In [ ]:
# ============================================================
"""
This block prepares the environment and parsing tools used throughout the BLE dataset analysis.
It imports the required Python libraries for file handling, data processing, and visualization,
sets paths to the MQTT CSV logs, and defines the BLE gateway MAC address. It also pre-compiles
regular expressions for extracting MAC addresses, 128-bit UUIDs, and hexadecimal fields from
the BLE logs. Finally, it configures a logging system to produce structured, timestamped
diagnostic messages.
"""
# ============================================================

from __future__ import annotations
import os
import json
import logging                 
import re                      # regex for MAC / UUID extraction
from collections import Counter
from typing import Any, List, Tuple, Optional, Sequence
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# -----------------------------
# Data paths
# -----------------------------

# Dataset path (update as needed)
DEFAULT_CSV: Path = Path("data") / "mqtt_input.csv"

# Gateway MAC (update as needed)
GATEWAY_MAC: str = "AA:BB:CC:DD:EE:FF"
GATEWAY_MAC = GATEWAY_MAC.upper()


# -----------------
# Regex definitions
# -----------------

# Generic MAC address pattern (AA:BB:CC:DD:EE:FF)
# Matches any MAC address inside BLE log lines.
MAC_RE = re.compile(r"([0-9A-Fa-f]{2}(?::[0-9A-Fa-f]{2}){5})")

# 128-bit UUID format used in BLE, 8-4-4-4-12 hex
UUID128_RE = re.compile(
    r"([0-9A-Fa-f]{8}-[0-9A-Fa-f]{4}-[0-9A-Fa-f]{4}-[0-9A-Fa-f]{4}-[0-9A-Fa-f]{12})"
)

# Single hexadecimal byte (00–FF)
# Used to validate TX Power, flags, and other 1-byte fields.
HEX_BYTE_RE = re.compile(r"^[0-9A-Fa-f]{2}$")


# -----------------------
# Logging configuration
# -----------------------

# INFO-level logging with timestamps for consistent diagnostic output.
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s] [%(levelname)s] %(message)s",
)

In [ ]:
# ============================================================

"""
This block loads the MQTT CSV file and converts the nested JSON payloads into a structured form
suitable for BLE analysis. The `load_csv()` function reads the file and reports how many MQTT 
events (rows) were recorded. The `safe_json_load()` function safely parses each payload string 
into a dictionary, ensuring that malformed JSON does not interrupt execution. The `parse_payload()` 
function applies this parser to every row and extracts two key fields: the internal event timestamp
(`log_timestamp`) and the raw bluetoothctl scan output (`revelations_raw`), which contains MAC addresses,
RSSI values, and other BLE data. 
"""

# ============================================================


def load_csv(file_path: str) -> pd.DataFrame:
    
    logging.info(f"Loading CSV file: {file_path}")
    df = pd.read_csv(file_path)
    logging.info(f"Loaded {len(df)} rows and columns: {df.columns.tolist()}")
    return df


def safe_json_load(text: Any) -> dict:
    if not isinstance(text, str):
        return {}
    try:
        return json.loads(text)
    except Exception:
        logging.debug(f"Failed to parse JSON: {text}")
        return {}


def parse_payload(df: pd.DataFrame) -> pd.DataFrame:
    if "payload" not in df.columns:
        raise KeyError("CSV does not contain required 'payload' column.")

    # Parse JSON payloads
    df["payload_json"] = df["payload"].apply(safe_json_load)

    # Extract relevant fields from JSON payload
    df["log_timestamp"] = df["payload_json"].apply(lambda x: x.get("timestamp"))
    df["revelations_raw"] = df["payload_json"].apply(lambda x: x.get("revelations"))

    logging.info("Parsed JSON payloads: extracted 'log_timestamp' and 'revelations_raw'.")
    logging.info(f"Unique payload timestamps: {df['log_timestamp'].nunique()}")

    return df


# Load CSV
df = load_csv(DEFAULT_CSV)

# Parse JSON payload inside it
df = parse_payload(df)

# Show the first few entries of timestamps (for validation)
print("\n--- SAMPLE TIMESTAMPS FROM PAYLOAD JSON ---")
print(df["log_timestamp"].head().to_string(index=False))



In [ ]:
# ============================================================

"""
This block inspects the raw MQTT metadata before further processing. It examines the `timestamp_client`
column to understand how MQTT message timing is recorded, checks the `topic` column to identify which 
MQTT topics are used, and inspects the `payload` column to view the raw JSON strings containing the BLE
scan data. This step clarifies the structure and content of the incoming MQTT messages and ensures that
the timestamp, topic, and payload formats are understood before parsing them in later stages.
"""

# ============================================================


# -------------------------------
# 1. Inspect timestamp_client
# -------------------------------
print(" TIMESTAMP_CLIENT (from MQTT client)")

ts_utc = pd.to_datetime(df["timestamp_client"], utc=True, errors="coerce")
ts_rome = ts_utc.dt.tz_convert("Europe/Rome")

print("Type (UTC parsed):", ts_utc.dtype)
print("Unique values:", ts_utc.nunique(dropna=True))

print("\nSample values:")
print(ts_rome.head(10).to_string(index=False))

# -------------------------------
# 2. Inspect topic column
# -------------------------------
print("\n\n TOPIC Column (MQTT topic names)")
print("Unique topics:", df["topic"].nunique())

print("\nList of all distinct topics:")
for t in df["topic"].unique():
    print(" -", t)

print("\nSample topic values:")
print(df["topic"].head(3).to_string(index=False))


# -------------------------------
# 3. Inspect payload column (raw JSON)
# -------------------------------
print("\n\n PAYLOAD Column (raw JSON strings)")
print("Showing first 5 payload entries:\n")

for i in range(3):
    print(f"[PAYLOAD {i}]")
    print(df["payload"].iloc[i])
    print("-" * 60)

In [ ]:
# ============================================================
#   Remove ANSI Codes (BLE-safe)
# Purpose:
#   - bluetoothctl adds ANSI color / cursor control codes
#   - These break regex parsing
#   - IMPORTANT:
#       • Do NOT remove raw control bytes (e.g. \x02)
#       • They may be part of real BLE payloads (iBeacon = 02 15)
# ============================================================

def clean_ansi(text: Any) -> Any:
    if not isinstance(text, str):
        return text

    # Remove ANSI escape sequences only
    text = re.sub(r"\x1B\[[0-?]*[ -/]*[@-~]", "", text)

    return text


# Apply cleaning to revelations_raw
df["revelations_clean"] = df["revelations_raw"].apply(clean_ansi)

logging.info("Created 'revelations_clean' by removing ANSI escape sequences only.")

# Optional verification output
print("\n===================================================")
print("   SAMPLE REVELATIONS (ANSI CLEANED)")
print("===================================================\n")

for i in range(min(2, len(df))):
    print(f"--- ROW {i} ---")
    print(df["revelations_raw"].iloc[i])
    print(f"--- ROW {i} ---")
    print(df["revelations_clean"].iloc[i])
    print("-" * 60)

In [ ]:
"""
Matches bracketed device event tags ([NEW], [CHG], [DEL]) even when they contain embedded ASCII control characters (0x00–0x1F)
or irregular whitespace, captures the valid event token, and normalizes malformed tags to their canonical form while preserving 
non-string inputs unchanged.
"""

CONTROL_TAG_RE = re.compile(r"\[\s*[\x00-\x1F]*\s*(NEW|CHG|DEL)\s*[\x00-\x1F]*\s*\]")

def normalize_device_tags(text: Any) -> Any:
    if not isinstance(text, str):
        return text

    # Normalize [\x01\x02CHG\x01\x02] → [CHG]
    return CONTROL_TAG_RE.sub(r"[\1]", text)

In [ ]:
df["revelations_clean"] = (
    df["revelations_raw"]
      .apply(clean_ansi)
      .apply(normalize_device_tags)
)

In [ ]:
# ============================================================

"""
This block analyzes controller-level activity in the Bluetooth logs by scanning the `revelations_clean`
column for patterns related to the BLE controller—such as `[CHG] Controller`, `Discovering: yes/no`, 
`Powered: yes/no`, and `Pairable: yes/no`. It counts how often each controller state occurs and determines 
how many MQTT messages contain any controller-related event. This helps describe the behaviour of the BLE 
scanner itself so that later analysis can clearly separate controller state changes from actual device-level 
events.
"""

# ============================================================

controller_patterns = {
    "[CHG] Controller": 0,
    "[NEW] Controller": 0,
    "[DEL] Controller": 0,
    "Discovering: yes": 0,
    "Discovering: no": 0,
    "Powered: yes": 0,
    "Powered: no": 0,
    "Pairable: yes": 0,
    "Pairable: no": 0,
}

total_rows = len(df)

for text in df["revelations_clean"]: #text = one block of text for one timestamp.
    if not isinstance(text, str):
        continue

    for pattern in controller_patterns.keys():
        if pattern in text:
            controller_patterns[pattern] += 1

# Count how many rows contain ANY controller keyword
def has_any_controller_event(text: str) -> bool:
    if not isinstance(text, str):
        return False
    if "Controller " in text:
        return True
    # also consider lines that only say "Discovering: ..." or "Powered: ..."
    for key in ["Discovering:", "Powered:", "Pairable:"]:
        if key in text:
            return True
    return False

rows_with_controller = sum(
    1 for txt in df["revelations_clean"] if has_any_controller_event(txt)
)

print("\n===================================================")
print("   CONTROLLER-LEVEL EVENTS ")
print("===================================================\n")

print(f"Total MQTT messages (rows): {total_rows}")
# print(f"Rows containing at least one controller-related event: {rows_with_controller}")
print()

print("Event counts (controller-level):")
for pattern, count in controller_patterns.items():
    print(f"  {pattern:<18} --> {count} occurrences")

In [ ]:
# ============================================================

"""
This block scans `revelations_clean` for `[NEW]`, `[CHG]`, and `[DEL]` device events, counts their
occurrences, and identifies how many MQTT messages contain device activity. `[NEW]` marks first 
detection, `[CHG]` updates an existing device, and `[DEL]` indicates removal or timeout. These events 
are later used to compute device entry/exit times and presence duration.
"""

# ============================================================

device_new_count = 0
device_chg_count = 0
device_del_count = 0

rows_with_device_events = 0

for text in df["revelations_clean"]:
    if not isinstance(text, str):
        continue

    has_device = False

    for line in text.splitlines():
        line_stripped = line.strip()
        if line_stripped.startswith("[NEW] Device "):
            device_new_count += 1
            has_device = True
        elif line_stripped.startswith("[CHG] Device "):
            device_chg_count += 1
            has_device = True
        elif line_stripped.startswith("[DEL] Device "):
            device_del_count += 1
            has_device = True

    if has_device:
        rows_with_device_events += 1

total_device_events = device_new_count + device_chg_count + device_del_count

print("\n===================================================")
print("   DEVICE-LEVEL EVENTS  ")
print("===================================================\n")

print(f"Rows containing at least one DEVICE event: {rows_with_device_events}")
print(f"Total DEVICE events (NEW + CHG + DEL):    {total_device_events}\n")

print("DEVICE-LEVEL EVENTS  type:")
print(f"  [NEW] Device  --> {device_new_count} events")
print(f"  [CHG] Device  --> {device_chg_count} events")
print(f"  [DEL] Device  --> {device_del_count} events")

In [ ]:
# ============================================================

"""
This block extracts all MAC addresses from the cleaned Bluetooth logs, flattens them into a single list, 
and counts how many times each MAC appears. It separates the gateway MAC from real BLE devices and produces 
a summary of how many unique devices were detected and how frequently each one appeared in the logs. 
"""

# ============================================================
#
# -----------------------------
# 1. Extract MAC list per row
# -----------------------------
def extract_mac_list(block_text: Any) -> list[str]:
    """
    Extract ALL MAC addresses appearing in a bluetoothctl block.
    
    This includes:
      - Device MACs (BLE devices)
      - Controller MAC (gateway)
    
    Filtering of gateway MAC will happen later.
    """
    if not isinstance(block_text, str):
        return []
    return [m.upper() for m in MAC_RE.findall(block_text)]


df["mac_list"] = df["revelations_clean"].apply(extract_mac_list)


# -----------------------------
# 2. Flatten into a single list
# -----------------------------
all_macs = []
for macs in df["mac_list"]:
    if isinstance(macs, list):
        all_macs.extend(macs)

# -----------------------------
# 3. Unique devices
# -----------------------------
unique_macs = sorted(set(all_macs))
total_unique = len(unique_macs)


# -----------------------------
# 4. Event counts per MAC
# -----------------------------
from collections import Counter
mac_event_counts = Counter(all_macs)


# -----------------------------
# 5. Identify gateway MAC
# -----------------------------
GATEWAY_MAC = "0C:9A:42:18:A5:A4"

gateway_event_count = mac_event_counts.get(GATEWAY_MAC, 0)

# Devices only = remove gateway
device_macs = [m for m in unique_macs if m != GATEWAY_MAC]
total_device_count = len(device_macs)


# -----------------------------
# 6. PRINT SUMMARY
# -----------------------------
print("\n===================================================")
print("      BLE DEVICE IDENTIFICATION ")
print("===================================================\n")

print(f"Total MAC addresses detected : {total_unique}")
print(f"Total devices (excluding gateway): {total_device_count}")
print(f"Gateway MAC ({GATEWAY_MAC}) detected with: {gateway_event_count} events\n")

print("\n===================================================")
print("     LIST OF DEVICES (MAC → Event Count)")
print("===================================================\n")

for mac, cnt in mac_event_counts.most_common():
    print(f"{mac:<18} ---------> {cnt} events")

print("\nTotal MACs printed:", len(mac_event_counts))

In [ ]:
# ============================================================
"""
Ensures a UTC timestamp column (df["ts_utc"]), extracts [NEW]/[CHG]/[DEL] device events with their MAC addresses, 
groups events by MAC into a dictionary of {timestamp, event_type, line}, sorts each MAC’s events,
and prints the complete per-device event history.
"""
# ============================================================

# Ensure a timestamp column exists (use ts_utc if already created; else create it)
if "ts_utc" not in df.columns:
    df = df.copy()
    df["ts_utc"] = pd.to_datetime(df["timestamp_client"], utc=True, errors="coerce")

mac_events_dict = {}  # MAC -> list of dicts {ts, event_type, line}

for ts, text in zip(df["ts_utc"].tolist(), df["revelations_clean"].tolist()):
    if not isinstance(text, str):
        continue

    for line in text.splitlines():
        line = line.strip()

        if line.startswith("[NEW] Device "):
            event_type = "NEW"
        elif line.startswith("[CHG] Device "):
            event_type = "CHG"
        elif line.startswith("[DEL] Device "):
            event_type = "DEL"
        else:
            continue

        m = MAC_RE.search(line)
        if not m:
            continue

        mac = m.group(1).upper()

        if mac not in mac_events_dict:
            mac_events_dict[mac] = []

        mac_events_dict[mac].append({
            "timestamp": ts,
            "event_type": event_type,
            "line": line,
        })

# Sort events per MAC by timestamp
for mac in mac_events_dict:
    mac_events_dict[mac] = sorted(
        mac_events_dict[mac],
        key=lambda x: (pd.Timestamp.min if pd.isna(x["timestamp"]) else x["timestamp"])
    )

print("\n===================================================")
print("   ALL DEVICE EVENTS GROUPED BY MAC (WITH TIMESTAMP)")
print("===================================================\n")

for mac, events in mac_events_dict.items():
    print(f"\n▶ MAC: {mac}   (Total events: {len(events)})")
    print("-" * 80)

    for e in events:
        ts = e.get("timestamp")
        ts_str = str(ts) if pd.notna(ts) else "NaT"

        line = e.get("line", "")
        print(f"{ts_str} | {line}")

In [ ]:
# ============================================================
"""
Eextract iBeacon advertisements by reconstructing ManufacturerData payload bytes, detecting valid iBeacon signatures, 
and decoding UUID,  Major, and Minor values; associates each decoded beacon with the current MAC address, returns (mac, uuid, major, minor) 
tuples per block, flattens and deduplicates them into mac_beacon_df based on full tuple uniqueness, and prints the total number of 
unique iBeacon identities and their details.
"""
# ============================================================

import re
from typing import Any, List, Tuple, Optional

# ------------------------------------------------------------
# Regex helpers
# ------------------------------------------------------------

HEX_IN_LINE_RE = re.compile(r"(?:0x)?([0-9A-Fa-f]{2})")

IBEACON_PATTERNS = [
    bytes.fromhex("4c 00 02 15"),
    bytes.fromhex("ff 4c 00 02 15"),
    bytes.fromhex("02 15"),
]

# ------------------------------------------------------------
# Helper functions
# ------------------------------------------------------------

def uuid_from_bytes(b: bytes) -> str:
    h = b.hex()
    return f"{h[0:8]}-{h[8:12]}-{h[12:16]}-{h[16:20]}-{h[20:32]}"

def parse_ibeacon_bytes(payload: bytes) -> Optional[Tuple[str, int, int]]:
    for pat in IBEACON_PATTERNS:
        idx = payload.find(pat)
        if idx == -1:
            continue

        start = idx + len(pat)
        if len(payload) < start + 16 + 2 + 2:
            continue

        uuid_b  = payload[start : start + 16]
        major_b = payload[start + 16 : start + 18]
        minor_b = payload[start + 18 : start + 20]

        return (
            uuid_from_bytes(uuid_b),
            int.from_bytes(major_b, "big"),
            int.from_bytes(minor_b, "big"),
        )
    return None


# ------------------------------------------------------------
# Main extractor
# ------------------------------------------------------------

def extract_ibeacon_tuples(block_text: Any) -> List[Tuple[str, str, int, int]]:
    """
    Returns a list of (mac, uuid, major, minor) extracted from one revelations text block.
    """
    results: List[Tuple[str, str, int, int]] = []
    if not isinstance(block_text, str):
        return results

    current_mac: Optional[str] = None
    collecting = False
    collected = bytearray()

    for line in block_text.splitlines():
        line = line.rstrip()

        # Track current MAC
        m = MAC_RE.search(line)
        if m:
            current_mac = m.group(1).upper()

        # Start ManufacturerData block
        if "ManufacturerData" in line and current_mac:
            collecting = True
            collected.clear()
            continue

        # End block on next event boundary
        if collecting and (line.startswith("[") or "Device " in line):
            parsed = parse_ibeacon_bytes(bytes(collected))
            if parsed and current_mac:
                uuid, major, minor = parsed
                results.append((current_mac, uuid, major, minor))
            collecting = False

        if collecting:
            # Printable hex bytes (NN, 0xNN, etc.)
            for h in HEX_IN_LINE_RE.finditer(line):
                collected.append(int(h.group(1), 16))

            # Raw control bytes (0x00–0x1F), preserved via latin-1
            raw = line.encode("latin1", errors="ignore")
            collected.extend(b for b in raw if b < 32)

    # Flush at end of block
    if collecting:
        parsed = parse_ibeacon_bytes(bytes(collected))
        if parsed and current_mac:
            uuid, major, minor = parsed
            results.append((current_mac, uuid, major, minor))

    return results


# ------------------------------------------------------------
# Apply extraction
# ------------------------------------------------------------

df["mac_beacon_tuples"] = df["revelations_clean"].apply(extract_ibeacon_tuples)

flat_rows = [t for row in df["mac_beacon_tuples"] for t in row]

# Keep full tuple uniqueness: (mac, uuid, major, minor)
mac_beacon_df = (
    pd.DataFrame(flat_rows, columns=["mac", "uuid", "major", "minor"])
      .drop_duplicates(subset=["mac", "uuid", "major", "minor"])
      .reset_index(drop=True)
)

# ------------------------------------------------------------
# PRINT RESULTS
# ------------------------------------------------------------

print("\n===================================================")
print("   iBEACON SUMMARY (MAC / UUID / MAJOR / MINOR)")
print("===================================================\n")

# Ensure DataFrame exists
if "mac_beacon_df" not in globals():
    raise NameError("mac_beacon_df is not defined.")

total_ibeacon_devices = len(mac_beacon_df)

print(f"Total iBeacon devices (MAC/UUID/Major/Minor): {total_ibeacon_devices}\n")

print("===================================================")
print("        iBEACON DEVICES (MAC / UUID / MAJOR / MINOR)")
print("===================================================\n")

if not mac_beacon_df.empty:
    print(mac_beacon_df.to_string(index=False))
else:
    print("No iBeacon devices found.")

In [ ]:
# =========================
"""
Builds a beacon-only presence table by filtering observed MAC addresses to those in mac_beacon_df, converting df["timestamp_client"]
to UTC timestamps, constructing a deduplicated (mac, ts) observation set from df["mac_list"], segmenting each MAC’s timeline into
sessions using a fixed inactivity gap threshold (GAP_SECONDS), and computing per-MAC first_seen, last_seen, number of sessions, 
total presence time (sum of session durations), and total absence time (sum of inter-observation gaps exceeding the threshold) 
with scalar outputs in seconds.
"""
# =========================
import pandas as pd

# -------------------------------------------------
# Config
# -------------------------------------------------
GAP_SECONDS = 180
GAP = pd.Timedelta(seconds=GAP_SECONDS)

# -------------------------------------------------
# Beacon MAC inventory 
# -------------------------------------------------
if "mac_beacon_df" not in globals() or mac_beacon_df.empty:
    raise ValueError("mac_beacon_df is missing or empty — no iBeacon MACs available")

beacon_macs = set(
    mac_beacon_df["mac"]
    .astype(str)
    .str.upper()
    .unique()
)

# -------------------------------------------------
# Parse timestamps
# -------------------------------------------------
df = df.copy()
df["ts_utc"] = pd.to_datetime(
    df["timestamp_client"], utc=True, errors="coerce"
)

# -------------------------------------------------
# Ensure mac_list exists
# -------------------------------------------------
if "mac_list" not in df.columns:
    def _extract_mac_list(block):
        if not isinstance(block, str):
            return []
        return [m.upper() for m in MAC_RE.findall(block)]

    df["mac_list"] = df["revelations_clean"].apply(_extract_mac_list)

# -------------------------------------------------
# Build beacon observations (MAC + timestamp)
# -------------------------------------------------
obs_rows = []

for ts, macs in zip(df["ts_utc"], df["mac_list"]):
    if pd.isna(ts) or not isinstance(macs, list):
        continue

    for mac in macs:
        mac_u = mac.upper()
        if mac_u in beacon_macs:
            obs_rows.append((mac_u, ts))

obs = (
    pd.DataFrame(obs_rows, columns=["mac", "ts"])
      .drop_duplicates()
      .sort_values(["mac", "ts"])
      .reset_index(drop=True)
)

# -------------------------------------------------
# Compute session-based metrics
# -------------------------------------------------
metrics = []

for mac, g in obs.groupby("mac", sort=False):
    t = g["ts"].sort_values().reset_index(drop=True)
    if t.empty:
        continue

    first_seen = t.iloc[0]
    last_seen  = t.iloc[-1]

    gaps = t.diff()

    # New session when gap > GAP
    is_new_session = gaps.isna() | (gaps > GAP)
    session_ids = is_new_session.cumsum()

    session_starts = t.groupby(session_ids).first()
    session_ends   = t.groupby(session_ids).last()

    session_durations = session_ends - session_starts

    total_presence = session_durations.sum()
    total_absence  = gaps[gaps > GAP].sum()

    metrics.append({
        "mac": mac,
        "first_seen": first_seen,
        "last_seen": last_seen,
        "num_sessions": int(len(session_durations)),
        "total_presence_seconds": float(total_presence.total_seconds()),
        "total_absence_seconds": float(total_absence.total_seconds()),
    })

# -------------------------------------------------
# Final table
# -------------------------------------------------
df_beacon_metrics = (
    pd.DataFrame(metrics)
      .sort_values(["first_seen", "mac"])
      .reset_index(drop=True)
)

# Human-friendly index
df_beacon_metrics.index = df_beacon_metrics.index + 1

df_beacon_metrics

In [ ]:
import pandas as pd

"""
Constructs per-MAC session intervals from an observation table containing ["mac", "ts"] by normalizing timestamps to datetime, 
removing invalid rows, ordering observations per MAC, and segmenting sequences into sessions using a fixed inactivity
threshold. For each session, computes session_start and session_end based on first and last timestamps within contiguous segments,
aggregates results into df_beacon_session_detail sorted, and prints the session timeline per MAC.
"""

# --- Safety: ensure ts is datetime ---
obs = obs.copy()
obs["ts"] = pd.to_datetime(obs["ts"], errors="coerce")
obs = obs.dropna(subset=["mac", "ts"])

session_rows = []

# --- Build per-session start/end for each MAC ---
for mac, g in obs.groupby("mac", sort=False):
    t = g["ts"].sort_values().reset_index(drop=True)
    if t.empty:
        continue

    gaps = t.diff()

    # New session when first row OR gap > GAP
    session_id = (gaps.isna() | (gaps > GAP)).cumsum()  # 1..N per MAC

    starts = t.groupby(session_id).first()
    ends   = t.groupby(session_id).last()

    for sid in starts.index:
        session_rows.append({
            "mac": mac,
            "session_id": int(sid),
            "session_start": starts.loc[sid],
            "session_end": ends.loc[sid],
        })

df_beacon_session_detail = (
    pd.DataFrame(session_rows)
      .sort_values(["mac", "session_id"])
      .reset_index(drop=True)
)

# --- Print per-MAC session list ---
for mac, g in df_beacon_session_detail.groupby("mac", sort=False):
    total_sessions = int(g["session_id"].max()) if not g.empty else 0
    print(f"\nMAC: {mac} | total_sessions: {total_sessions}")
    for _, r in g.iterrows():
        print(f"  Session {int(r['session_id'])}: {r['session_start']} | {r['session_end']}")

In [ ]:
import pandas as pd

# =========================
"""
Constructs a session-level observation dataset restricted to iBeacon MAC addresses by normalizing timestamps to UTC, 
validating and filtering usable rows, standardizing mac_list entries, exploding per-row MAC observations, and retaining 
only MACs present in beacon_macs. Observations are deduplicated and chronologically ordered per MAC, segmented into sessions
using a fixed inactivity threshold, and assigned incremental session_id values. For each observation, only log lines
referencing the corresponding MAC are retained. The code produces both a detailed session-observation table and an aggregated 
session summary, and prints the full per-session timeline including timestamp and MAC-specific payload content.
"""
# =========================

# --- Ensure GAP is a Timedelta
if not isinstance(GAP, pd.Timedelta):
    GAP = pd.to_timedelta(GAP)

# --- Copy + parse timestamps (UTC) ---
df = df.copy()
df["ts_utc"] = pd.to_datetime(df["timestamp_client"], utc=True, errors="coerce")
df = df[df["ts_utc"].notna() & df["revelations_clean"].notna()].copy()

# --- Normalize beacon_macs to uppercase string ---
beacon_macs = {str(m).upper() for m in beacon_macs if pd.notna(m)}

# --- Normalize mac_list ---
def _normalize_mac_list(x):
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return []
    if isinstance(x, list):
        return x
    # if it's a string, try splitting on common separators
    if isinstance(x, str):
        s = x.strip()
        if not s:
            return []
        # handles "['AA', 'BB']"; keep simple
        for sep in ["|", ";", ",", " "]:
            if sep in s:
                parts = [p for p in (pp.strip() for pp in s.split(sep)) if p]
                if len(parts) > 1:
                    return parts
        return [s]
    return [x]

df["mac_list_norm"] = df["mac_list"].apply(_normalize_mac_list)

# --- Each observation keeps the revelations_clean from that MQTT row ---
obs_detail = (
    df[["ts_utc", "revelations_clean", "mac_list_norm"]]
      .explode("mac_list_norm")
      .rename(columns={"mac_list_norm": "mac"})
)

# --- Normalize MAC + filter to iBeacon MACs only ---
obs_detail["mac"] = obs_detail["mac"].astype(str).str.upper().str.strip()
obs_detail = obs_detail[obs_detail["mac"].isin(beacon_macs)].copy()

# --- Drop duplicates + sort ---
obs_detail = (
    obs_detail.drop_duplicates(subset=["mac", "ts_utc"])
              .sort_values(["mac", "ts_utc"])
              .reset_index(drop=True)
)

# --- Assign session_id per MAC after GAP ---
def _assign_sessions(group: pd.DataFrame) -> pd.DataFrame:
    g = group.sort_values("ts_utc").copy()
    gaps = g["ts_utc"].diff()
    g["session_id"] = (gaps.isna() | (gaps > GAP)).cumsum().astype(int)  # 1..N
    return g

df_beacon_session_obs = (
    obs_detail.groupby("mac", group_keys=False, sort=False)
              .apply(_assign_sessions)
              .reset_index(drop=True)
)

# --- Keep only lines that mention this MAC ---
def _keep_mac_lines(block, mac: str) -> str:
    if not isinstance(block, str):
        return ""
    mac_u = str(mac).upper()
    out = []
    for ln in block.splitlines():
        if mac_u in ln.upper():
            out.append(ln)
    return "\n".join(out)

df_beacon_session_obs["revelations_clean_mac_only"] = df_beacon_session_obs.apply(
    lambda r: _keep_mac_lines(r["revelations_clean"], r["mac"]),
    axis=1
)

# --- Session summary (start/end/count) ---
df_beacon_session_summary = (
    df_beacon_session_obs.groupby(["mac", "session_id"], as_index=False)
    .agg(
        session_start=("ts_utc", "min"),
        session_end=("ts_utc", "max"),
        num_observations=("ts_utc", "count"),
    )
    .sort_values(["mac", "session_id"])
    .reset_index(drop=True)
)

# --- PRINT: per MAC -> per session -> each timestamp + payload text ---
TEXT_FIELD = "revelations_clean_mac_only"

for mac, gmac in df_beacon_session_obs.groupby("mac", sort=False):
    total_sessions = int(gmac["session_id"].max()) if not gmac.empty else 0
    print(f"\nMAC: {mac} | total_sessions: {total_sessions}")

    for sid, gs in gmac.groupby("session_id", sort=True):
        s_start = gs["ts_utc"].min()
        s_end   = gs["ts_utc"].max()
        print(f"  Session {int(sid)}: {s_start} -> {s_end} | observations: {len(gs)}")

        for _, r in gs.iterrows():
            print(f"    - {r['ts_utc']}")
            txt = r[TEXT_FIELD]
            print(txt if isinstance(txt, str) else "")
            print("")

In [ ]:
"""
Implements a Tkinter GUI to inspect iBeacon tuples, session presence, and raw log analytics by MAC address; validates and normalizes 
input DataFrames, applies a global MAC filter across tabs, displays summary tables and session event text, computes per-MAC RSSI 
samples and binned NEW/CHG/DEL counts with caching, classifies proximity and movement from RSSI using thresholds and 
MAD-based rolling dispersion, embeds Matplotlib plots with consistent styling, and exports the selected plot image and its source data.
"""
from matplotlib.figure import Figure
import matplotlib.dates as mdates
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog, messagebox
from pathlib import Path
import pandas as pd
import re
import numpy as np

import matplotlib
matplotlib.use("TkAgg")

# ---- Universal plot defaults (applies to all figures) ----
IEEE_WIDTH = 3.5   # inches (single column)
IEEE_HEIGHT = 2.5  # inches 

matplotlib.rcParams.update({
    "font.family": "serif",
    "font.serif": ["Times New Roman", "Times", "STIXGeneral", "TeX Gyre Termes"],
    "mathtext.fontset": "stix",
    "axes.unicode_minus": False,
    "pdf.use14corefonts": True,

    "axes.labelsize": 9,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "legend.fontsize": 6,
    "axes.titlesize": 9,

    "axes.linewidth": 0.9,
    "lines.linewidth": 1.4,
    "grid.linewidth": 0.5,
    "xtick.major.width": 0.6,
    "ytick.major.width": 0.6,
})

# Proximity thresholds (RSSI in dBm)
NEAR_RSSI_THRESHOLD = -50
FAR_RSSI_THRESHOLD = -75

# Movement thresholds
STATIC_STD_MAX = 3.0
MOVING_STD_MIN = 4.0
MOVING_STEP_MIN = 3.5

# Analytics defaults
EVENT_BIN_FREQ = "5min" 
ROLLING_WINDOW = 30

EVENT_TAG_RE = re.compile(r"^\s*\[(NEW|CHG|DEL)\]", re.MULTILINE)
RSSI_RE = re.compile(r"RSSI\s*:\s*(-?\d+)", re.IGNORECASE)

from zoneinfo import ZoneInfo
UTC_TZ = ZoneInfo("UTC")


def _binfreq_minutes(freq: str) -> int:
    """
    Convert pandas 'T' minute-based frequency strings to integer minutes for labels.
    """
    if not isinstance(freq, str):
        return None
    m = re.fullmatch(r"\s*(\d+)\s*T\s*", freq, flags=re.IGNORECASE)
    return int(m.group(1)) if m else None


class BeaconDeepViewer(tk.Tk):
    _RSSI_RE = re.compile(r"RSSI\s*:\s*(-?\d+)", re.IGNORECASE)

    def __init__(self, df_tuples: pd.DataFrame, df_sessions: pd.DataFrame, df_raw: pd.DataFrame):
        super().__init__()
        self.title("iBeacon Session Inspector")
        self.geometry("1300x860")
        self.minsize(1100, 720)
        self.resizable(True, True)

        self.df_tuples = df_tuples.copy()
        self.df_sessions = df_sessions.copy()
        self.df_raw = df_raw.copy()

        self.uuid_col = None
        self._current_mac = None

        # Global filter state
        self._filter_text = ""
        self._filter_upper = ""
        self._all_macs = []
        self._ui_ready = False

        # Analytics state
        self._analytics_cache = {} 
        
        self._analytics_tmin = None
        self._analytics_tmax = None

        self.TIME_AXIS_LABEL = "Observation Timestamp (UTC)"
        self.TIME_TICK_FMT = "%H:%M"
        self.EVENT_BIN_MIN = 5
        self.EVENT_BIN_LABEL = f"{self.EVENT_BIN_MIN} min"
        self._plot_data = {} 



        # Legend style 
        self.LEGEND_KW = {
            "loc": "upper right",
            "frameon": True,
            "fontsize": 6,
            "borderaxespad": 0.4,
            "handlelength": 1.5,
            "handletextpad": 0.4,
        }
        
        self.PLOT_STYLE = {
            "title_size": 9,
            "label_size": 9,
            "tick_size": 8,
            "legend_size": self.LEGEND_KW["fontsize"],
        }

        # ---------------------------------------------------------
        # Universal plot style controls
        # ---------------------------------------------------------

        self._current_plot_name = None
        self._current_plot_mac = None
        self._current_plot_df = None 

        self._prepare_inputs()
        self._build_ui()
        self._ui_ready = True
        self._apply_filter("")

    # ---------------------------------------------------------
    # Data preparation
    # ---------------------------------------------------------
    @staticmethod
    def _normalize_mac_list(x):
        if x is None or (isinstance(x, float) and pd.isna(x)):
            return []
        if isinstance(x, (list, tuple, set)):
            return [str(m).upper().strip() for m in x if pd.notna(m) and str(m).strip()]
        if isinstance(x, str):
            s = x.strip()
            if not s:
                return []
            for sep in ["|", ";", ",", " "]:
                if sep in s:
                    parts = [p.strip() for p in s.split(sep) if p.strip()]
                    if len(parts) > 1:
                        return [p.upper() for p in parts]
            return [s.upper()]
        return [str(x).upper().strip()]

    def _prepare_inputs(self):
        # ---- tuples
        if "mac" not in self.df_tuples.columns:
            raise KeyError("df_tuples must contain 'mac'")
    
        if "beacon_uuid" in self.df_tuples.columns:
            self.uuid_col = "beacon_uuid"
        elif "uuid" in self.df_tuples.columns:
            self.uuid_col = "uuid"
        else:
            raise KeyError("df_tuples must contain 'beacon_uuid' or 'uuid'")
    
        for c in ("major", "minor"):
            if c not in self.df_tuples.columns:
                raise KeyError(f"df_tuples must contain '{c}'")
    
        self.df_tuples["mac"] = self.df_tuples["mac"].astype(str).str.upper().str.strip()
        self.df_tuples[self.uuid_col] = self.df_tuples[self.uuid_col].astype(str).str.strip().str.lower()
        self.df_tuples["major"] = pd.to_numeric(self.df_tuples["major"], errors="coerce")
        self.df_tuples["minor"] = pd.to_numeric(self.df_tuples["minor"], errors="coerce")
    
        self.df_tuples = self.df_tuples.dropna(subset=["mac", self.uuid_col, "major", "minor"]).copy()
        self.df_tuples["major"] = self.df_tuples["major"].astype(int)
        self.df_tuples["minor"] = self.df_tuples["minor"].astype(int)
    
        self.df_tuples = (
            self.df_tuples
            .drop_duplicates(subset=["mac", self.uuid_col, "major", "minor"])
            .sort_values(["mac", self.uuid_col, "major", "minor"])
            .reset_index(drop=True)
        )
    
        self._list_macs = sorted(self.df_tuples["mac"].unique().tolist())
        self._list_uuids = sorted(self.df_tuples[self.uuid_col].unique().tolist())
        self._list_majors = sorted(self.df_tuples["major"].unique().tolist())
        self._list_minors = sorted(self.df_tuples["minor"].unique().tolist())
        self._all_macs = list(self._list_macs)
    
        # ---- sessions 
        required = {"mac", "session_id", "session_start", "session_end"}
        if not required.issubset(self.df_sessions.columns):
            raise KeyError(f"df_sessions must contain {required}")
    
        self.df_sessions["mac"] = self.df_sessions["mac"].astype(str).str.upper().str.strip()
    
        s0 = pd.to_datetime(self.df_sessions["session_start"], errors="coerce")
        s1 = pd.to_datetime(self.df_sessions["session_end"], errors="coerce")
    
        # If tz-naive, assume already UTC
        self.df_sessions["session_start"] = (
            s0.dt.tz_localize("UTC", nonexistent="shift_forward", ambiguous="NaT")
            if s0.dt.tz is None else s0.dt.tz_convert("UTC")
        )
        self.df_sessions["session_end"] = (
            s1.dt.tz_localize("UTC", nonexistent="shift_forward", ambiguous="NaT")
            if s1.dt.tz is None else s1.dt.tz_convert("UTC")
        )
    
        self.df_sessions = self.df_sessions.dropna(subset=["session_start", "session_end"]).copy()
    
        if "revelations_clean" not in self.df_raw.columns:
            raise KeyError("df_raw must contain 'revelations_clean'")
        if "mac_list" not in self.df_raw.columns:
            raise KeyError("df_raw must contain 'mac_list'")
    
        ROME_TZ = ZoneInfo("Europe/Rome")
    
        if "ts_utc" in self.df_raw.columns:
            ts = pd.to_datetime(self.df_raw["ts_utc"], errors="coerce")
            self.df_raw["ts_utc"] = (
                ts.dt.tz_localize("UTC", nonexistent="shift_forward", ambiguous="NaT")
                if ts.dt.tz is None else ts.dt.tz_convert("UTC")
            )
        else:
            if "timestamp_client" in self.df_raw.columns:
                ts = pd.to_datetime(self.df_raw["timestamp_client"], errors="coerce")
                self.df_raw["ts_utc"] = (
                    ts.dt.tz_localize(ROME_TZ, nonexistent="shift_forward", ambiguous="NaT").dt.tz_convert("UTC")
                    if ts.dt.tz is None else ts.dt.tz_convert("UTC")
                )
            else:
                raise KeyError("df_raw must contain 'ts_utc' or 'timestamp_client'")
    
        self.df_raw = self.df_raw.dropna(subset=["ts_utc"]).copy()
        self.df_raw["mac_list"] = self.df_raw["mac_list"].apply(self._normalize_mac_list)

    # ---------------------------------------------------------
    # Global filter
    # ---------------------------------------------------------
    def _apply_filter(self, text: str):
        text = (text or "").strip()
        self._filter_text = text
        self._filter_upper = text.upper()
        if self._ui_ready:
            self._refresh_all_tabs()

    def _filtered_macs(self):
        if not self._filter_upper:
            return list(self._all_macs)
        return [m for m in self._all_macs if self._filter_upper in m]

    # ---------------------------------------------------------
    # iBeacon Summary text 
    # ---------------------------------------------------------
    def _iBeacon_text_filtered(self, df_subset: pd.DataFrame) -> str:
        lines = []
        lines.append("iBeacon Summary\n")

        if df_subset.empty:
            lines.append("No records for current filter.")
            return "\n".join(lines)

        list_macs = sorted(df_subset["mac"].unique().tolist())
        list_uuids = sorted(df_subset[self.uuid_col].unique().tolist())
        list_majors = sorted(df_subset["major"].unique().tolist())
        list_minors = sorted(df_subset["minor"].unique().tolist())

        lines.append(f"Active MAC filter: {self._filter_text!r}" if self._filter_text else "Active MAC filter: (none)")
        lines.append("")
        lines.append(f"Total unique iBeacon identities (MAC|UUID|Major|Minor): {len(df_subset)}")
        lines.append(f"Unique MACs          : {len(list_macs)}")
        lines.append(f"Unique UUIDs         : {len(list_uuids)}")
        lines.append(f"Unique Majors        : {len(list_majors)}")
        lines.append(f"Unique Minors        : {len(list_minors)}")
        lines.append("")

        lines.append("---- MAC Addresses ----")
        lines.extend(f"  {m}" for m in list_macs)
        lines.append("")

        lines.append("---- UUIDs ----")
        lines.extend(f"  {u}" for u in list_uuids)
        lines.append("")

        lines.append("---- Majors ----")
        lines.extend(f"  {m}" for m in list_majors)
        lines.append("")

        lines.append("---- Minors ----")
        lines.extend(f"  {m}" for m in list_minors)

        return "\n".join(lines)

    # ---------------------------------------------------------
    # Parsing 
    # ---------------------------------------------------------
    @classmethod
    def _parse_rssi_from_text(cls, text: str):
        if not isinstance(text, str):
            return []
        vals = []
        for m in cls._RSSI_RE.finditer(text):
            try:
                vals.append(int(m.group(1)))
            except Exception:
                pass
        return vals

    # ---------------------------------------------------------
    # Movement detection (MAD-based)
    # ---------------------------------------------------------
    @staticmethod
    def _detect_moving_or_static_mad(rssi_values):
        rssi_values = [int(v) for v in (rssi_values or []) if v is not None]
        n = len(rssi_values)
        if n < 5:
            return ("STATIC", None, None, n)

        r = np.array(rssi_values, dtype=float)

        med = float(np.median(r))
        mad = float(np.median(np.abs(r - med)))
        std = 1.4826 * mad

        # Keep step only as a *reported* metric
        median_step = float(np.median(np.abs(np.diff(r)))) if n >= 2 else 0.0

        # Decision based primarily on MAD-std (robust spread)
        if std <= STATIC_STD_MAX:
            return ("STATIC", std, median_step, n)

        if std >= MOVING_STD_MIN:
            return ("MOVING", std, median_step, n)

        # MID-zone: step is only a tie-breaker
        if median_step >= MOVING_STEP_MIN:
            return ("MOVING", std, median_step, n)

        return ("STATIC", std, median_step, n)

    # ---------------------------------------------------------
    # Proximity label
    # ---------------------------------------------------------
    @staticmethod
    def _proximity_label(rssi: int) -> str:
        try:
            r = int(rssi)
        except Exception:
            return None
        if r >= NEAR_RSSI_THRESHOLD:
            return "NEAR"
        if r <= FAR_RSSI_THRESHOLD:
            return "FAR"
        return "MID"

    # ---------------------------------------------------------
    # MAC-scoped event extraction 
    # ---------------------------------------------------------
    @staticmethod
    def extract_events_for_mac(block: str, mac: str) -> str:
        if not isinstance(block, str):
            return ""
        mac = mac.upper()
        out = []
        capturing = False
        for ln in block.splitlines():
            if ln.lstrip().startswith("["):
                if mac in ln.upper():
                    capturing = True
                    out.append(ln)
                else:
                    capturing = False
            else:
                if capturing:
                    out.append(ln)
        return "\n".join(out).strip()

    # ---------------------------------------------------------
    # Presence summary computation 
    # ---------------------------------------------------------
    @staticmethod
    def _fmt_tdelta_hms(td) -> str:
        if td is None or pd.isna(td):
            return ""
        try:
            total_seconds = int(td.total_seconds())
        except Exception:
            return ""
        sign = "-" if total_seconds < 0 else ""
        total_seconds = abs(total_seconds)
        h = total_seconds // 3600
        m = (total_seconds % 3600) // 60
        s = total_seconds % 60
        return f"{sign}{h}:{m:02d}:{s:02d}"

    def _compute_presence_summary_filtered(self, macs):
        if self.df_sessions.empty or not macs:
            return pd.DataFrame(columns=[
                "mac", "first_seen", "last_seen",
                "number_of_sessions", "total_present_time", "total_absent_time"
            ])

        ds = self.df_sessions[self.df_sessions["mac"].isin(macs)].copy()

        out_rows = []
        for mac, g in ds.groupby("mac", sort=False):
            g = g.sort_values("session_start").reset_index(drop=True)
            first_seen = g["session_start"].min()
            last_seen = g["session_end"].max()
            num_sessions = int(g.shape[0])
            present = (g["session_end"] - g["session_start"]).sum()

            gaps = (g["session_start"].shift(-1) - g["session_end"]).dropna()
            gaps = gaps[gaps > pd.Timedelta(0)]
            absent = gaps.sum() if not gaps.empty else pd.Timedelta(0)

            out_rows.append({
                "mac": mac,
                "first_seen": first_seen,
                "last_seen": last_seen,
                "number_of_sessions": num_sessions,
                "total_present_time": present,
                "total_absent_time": absent
            })

        dfp = pd.DataFrame(out_rows)
        return dfp.sort_values("mac").reset_index(drop=True) if not dfp.empty else dfp

    # ---------------------------------------------------------
    # Analytics dataset (MAC-scoped RSSI + binned event counts)
    # ---------------------------------------------------------
    def _build_analytics_dataset(self, mac: str):
        mac = (mac or "").upper().strip()
        if not mac:
            return pd.DataFrame(), pd.DataFrame()

        if mac in self._analytics_cache:
            return self._analytics_cache[mac]

        mask = self.df_raw["mac_list"].apply(lambda xs: mac in (xs or []))
        rows = self.df_raw.loc[mask].sort_values("ts_utc")

        samples = []
        event_rows = []

        for _, rr in rows.iterrows():
            ts = rr["ts_utc"]
            block = rr["revelations_clean"]
            mac_scoped = self.extract_events_for_mac(block, mac)
            text_to_parse = mac_scoped if mac_scoped else block

            new_cnt = chg_cnt = del_cnt = 0
            if mac_scoped:
                for t in EVENT_TAG_RE.findall(mac_scoped):
                    if t == "NEW":
                        new_cnt += 1
                    elif t == "CHG":
                        chg_cnt += 1
                    elif t == "DEL":
                        del_cnt += 1

            rssi_vals = []
            if isinstance(text_to_parse, str):
                for m in RSSI_RE.finditer(text_to_parse):
                    try:
                        rssi_vals.append(int(m.group(1)))
                    except Exception:
                        pass

            for v in rssi_vals:
                prox = self._proximity_label(v)
                if prox is None:
                    continue
                samples.append({"ts_utc": ts, "rssi": int(v), "prox": prox})

            event_rows.append({
                "ts_utc": ts,
                "NEW": int(new_cnt),
                "CHG": int(chg_cnt),
                "DEL": int(del_cnt),
                "rssi_samples": int(len(rssi_vals)),
            })

        df_s = pd.DataFrame(samples)
        df_e = pd.DataFrame(event_rows)

        if not df_e.empty:
            binned = (
                df_e.sort_values("ts_utc")
                   .set_index("ts_utc")
                   .resample(EVENT_BIN_FREQ)
                   .sum(numeric_only=True)
                   .reset_index()
            )
        else:
            binned = pd.DataFrame(columns=["ts_utc", "NEW", "CHG", "DEL", "rssi_samples"])

        self._analytics_cache[mac] = (df_s, binned)
        return df_s, binned

    # ---------------------------------------------------------
    # Movement Timeline plot
    # ---------------------------------------------------------
    @staticmethod
    def _mad_std(arr: np.ndarray) -> float:
        arr = arr.astype(float)
        med = np.median(arr)
        mad = np.median(np.abs(arr - med))
        return 1.4826 * mad

    def _rolling_movement(self, rssi_series: pd.Series, window: int = ROLLING_WINDOW) -> pd.DataFrame:
        r = rssi_series.dropna().astype(float)
        if r.shape[0] < max(5, window):
            return pd.DataFrame(columns=["roll_mad_std", "label"])

        roll_mad_std = r.rolling(window).apply(lambda x: self._mad_std(np.array(x)), raw=False)

        def _med_step(x):
            x = np.array(x, dtype=float)
            if x.size < 2:
                return 0.0
            return float(np.median(np.abs(np.diff(x))))

        roll_step = r.rolling(window).apply(_med_step, raw=False)

        labels = []
        for std, step in zip(roll_mad_std.values, roll_step.values):
            if np.isnan(std) or np.isnan(step):
                labels.append(None)
                continue

            if std <= STATIC_STD_MAX:
                labels.append("STATIC")
            elif std >= MOVING_STD_MIN:
                labels.append("MOVING")
            elif step >= MOVING_STEP_MIN:
                labels.append("MOVING")
            else:
                labels.append("STATIC")

        return pd.DataFrame({"roll_mad_std": roll_mad_std, "label": labels}, index=r.index)

    # ---------------------------------------------------------
    # Universal axis styling + exporting
    # ---------------------------------------------------------
    def _apply_universal_axis_style(self, ax):
        ax.title.set_fontsize(self.PLOT_STYLE["title_size"])
        ax.xaxis.label.set_fontsize(self.PLOT_STYLE["label_size"])
        ax.yaxis.label.set_fontsize(self.PLOT_STYLE["label_size"])
        ax.tick_params(axis="both", which="major", labelsize=self.PLOT_STYLE["tick_size"])
        ax.tick_params(axis="both", which="minor", labelsize=self.PLOT_STYLE["tick_size"])
    # ---------------------------------------------------------------------------

    def _export_current_plot_image(self):
        if not self._current_plot_name or self._current_plot_name not in self._plot_tabs:
            messagebox.showwarning("Export plot", "No plot is currently available to export.")
            return

        p = self._plot_tabs[self._current_plot_name]
        fig = p["fig"]

        default_base = f"{self._current_plot_mac or 'MAC'}_{self._current_plot_name}".replace(" ", "_")
        file_path = filedialog.asksaveasfilename(
            title="Save plot image",
            defaultextension=".pdf",
            initialfile=default_base,
            filetypes=[
                ("PDF document", "*.pdf"),
                ("PNG image", "*.png"),
                ("JPEG image", "*.jpg"),
            ],
        )
        if not file_path:
            return

        try:
            fig.savefig(file_path, dpi=300)
            messagebox.showinfo("Export plot", f"Saved:\n{file_path}")
        except Exception as e:
            messagebox.showerror("Export plot", f"Failed to export plot:\n{e}")

    def _export_current_plot_data(self):
        if self._current_plot_df is None or not isinstance(self._current_plot_df, pd.DataFrame):
            messagebox.showwarning("Export data", "No plot data is currently available to export.")
            return

        default_base = f"{self._current_plot_mac or 'MAC'}_{self._current_plot_name}_data".replace(" ", "_")
        file_path = filedialog.asksaveasfilename(
            title="Save plot data",
            defaultextension=".csv",
            initialfile=default_base,
            filetypes=[
                ("CSV", "*.csv"),
                ("Excel", "*.xlsx"),
            ],
        )
        if not file_path:
            return

        try:
            suffix = Path(file_path).suffix.lower()

            if suffix == ".xlsx":
                df_out = self._current_plot_df.copy()

                # Excel cannot handle timezone-aware datetimes
                for col in df_out.select_dtypes(include=["datetimetz"]).columns:
                    df_out[col] = df_out[col].dt.tz_localize(None)

                df_out.to_excel(file_path, index=False)
            else:
                self._current_plot_df.to_csv(file_path, index=False)

            messagebox.showinfo("Export data", f"Saved:\n{file_path}")

        except Exception as e:
            messagebox.showerror("Export data", f"Failed to export data:\n{e}")
            

    def _set_current_plot_context(self, plot_name: str, mac: str, df: pd.DataFrame):
        self._current_plot_mac = mac
        self._plot_data[plot_name] = df.copy() if isinstance(df, pd.DataFrame) else None
        
        if self._current_plot_name == plot_name:
            self._current_plot_df = self._plot_data[plot_name]
    
    def _compute_common_time_range(self, df_s: pd.DataFrame, df_bins: pd.DataFrame):
        tmin = tmax = None
    
        if isinstance(df_s, pd.DataFrame) and not df_s.empty and "ts_utc" in df_s.columns:
            s = pd.to_datetime(df_s["ts_utc"], utc=True, errors="coerce").dropna()
            if not s.empty:
                tmin, tmax = s.min(), s.max()
    
        if isinstance(df_bins, pd.DataFrame) and not df_bins.empty and "ts_utc" in df_bins.columns:
            b = pd.to_datetime(df_bins["ts_utc"], utc=True, errors="coerce").dropna()
            if not b.empty:
                if tmin is None or b.min() < tmin:
                    tmin = b.min()
                if tmax is None or b.max() > tmax:
                    tmax = b.max()
    
        self._analytics_tmin, self._analytics_tmax = tmin, tmax
    
    
    def _apply_common_time_xlim(self, ax):
        if self._analytics_tmin is None or self._analytics_tmax is None:
            return
        if self._analytics_tmax >= self._analytics_tmin:
            ax.set_xlim(self._analytics_tmin, self._analytics_tmax)
    # =========================================================
    
    def _on_plot_tab_changed(self, event=None):
        try:
            tab_id = self.plot_nb.select()
            tab_text = self.plot_nb.tab(tab_id, "text")
            self._current_plot_name = tab_text
            self._current_plot_df = self._plot_data.get(tab_text)
        except Exception:
            pass


    # ---------------------------------------------------------
    # UI: Tabs (Global filter)
    # ---------------------------------------------------------
    def _build_ui(self):
        filter_bar = ttk.Frame(self)
        filter_bar.pack(fill="x", padx=10, pady=(10, 6))

        ttk.Label(filter_bar, text="Global MAC filter:").pack(side="left")

        self.filter_var = tk.StringVar()
        self.filter_entry = ttk.Entry(filter_bar, textvariable=self.filter_var, width=40)
        self.filter_entry.pack(side="left", padx=8)
        self.filter_entry.bind("<Return>", lambda e: self._apply_filter(self.filter_var.get()))

        ttk.Button(filter_bar, text="Apply", command=lambda: self._apply_filter(self.filter_var.get())).pack(side="left")
        ttk.Button(filter_bar, text="Clear", command=self._on_clear_filter).pack(side="left", padx=6)

        self.filter_status = ttk.Label(filter_bar, text="(no filter)")
        self.filter_status.pack(side="left", padx=12)

        nb = ttk.Notebook(self)
        nb.pack(fill="both", expand=True)

        self.tab_iBeacon = ttk.Frame(nb)
        self.tab_presence = ttk.Frame(nb)
        self.tab_inspect = ttk.Frame(nb)
        self.tab_track = ttk.Frame(nb)
        self.tab_analytics = ttk.Frame(nb)

        nb.add(self.tab_iBeacon, text="iBeacon Summary")
        nb.add(self.tab_presence, text="Presence Summary")
        nb.add(self.tab_inspect, text="Session Inspector")
        nb.add(self.tab_track, text="Track Devices")
        nb.add(self.tab_analytics, text="Plots")

        self._build_tab_iBeacon(self.tab_iBeacon)
        self._build_tab_presence_summary(self.tab_presence)
        self._build_tab_session_inspector(self.tab_inspect)
        self._build_tab_track_devices(self.tab_track)
        self._build_tab_analytics(self.tab_analytics)

    def _on_clear_filter(self):
        self.filter_var.set("")
        self._apply_filter("")

    # ---------------------------------------------------------
    # Tab 1: iBeacon Summary
    # ---------------------------------------------------------
    def _build_tab_iBeacon(self, parent):
        paned = ttk.Panedwindow(parent, orient="vertical")
        paned.pack(fill="both", expand=True, padx=10, pady=10)

        top = ttk.Frame(paned)
        bottom = ttk.Frame(paned)
        paned.add(top, weight=2)
        paned.add(bottom, weight=3)

        txt_box = ttk.LabelFrame(top, text="iBeacon Summary")
        txt_box.pack(fill="both", expand=True)

        self.iBeacon_text = tk.Text(txt_box, wrap="word")
        self.iBeacon_text.pack(fill="both", expand=True)
        self.iBeacon_text.configure(state="disabled")

        tbl_box = ttk.LabelFrame(bottom, text="Tuples (MAC | UUID | Major | Minor)")
        tbl_box.pack(fill="both", expand=True)

        self.inv_table = ttk.Treeview(
            tbl_box, columns=("mac", "uuid", "major", "minor"),
            show="headings", height=16
        )
        for col in ("mac", "uuid", "major", "minor"):
            self.inv_table.heading(col, text=col)

        self.inv_table.column("mac", anchor="w", width=220, stretch=False)
        self.inv_table.column("uuid", anchor="w", width=520, stretch=True)
        self.inv_table.column("major", anchor="center", width=90, stretch=False)
        self.inv_table.column("minor", anchor="center", width=90, stretch=False)

        yscroll = ttk.Scrollbar(tbl_box, orient="vertical", command=self.inv_table.yview)
        self.inv_table.configure(yscrollcommand=yscroll.set)

        self.inv_table.pack(side="left", fill="both", expand=True)
        yscroll.pack(side="right", fill="y")

    def _refresh_tab_ibeacon(self, macs):
        df_sub = self.df_tuples[self.df_tuples["mac"].isin(macs)].copy()

        self.iBeacon_text.configure(state="normal")
        self.iBeacon_text.delete("1.0", "end")
        self.iBeacon_text.insert("1.0", self._iBeacon_text_filtered(df_sub))
        self.iBeacon_text.configure(state="disabled")

        self.inv_table.delete(*self.inv_table.get_children())
        for _, r in df_sub.iterrows():
            self.inv_table.insert("", "end", values=(r["mac"], r[self.uuid_col], int(r["major"]), int(r["minor"])))

    # ---------------------------------------------------------
    # Tab 2: Presence Summary
    # ---------------------------------------------------------
    def _build_tab_presence_summary(self, parent):
        box = ttk.LabelFrame(parent, text="iBeacon Presence Summary (from sessions)")
        box.pack(fill="both", expand=True, padx=10, pady=10)

        self.pres_table = ttk.Treeview(
            box, columns=("mac", "first", "last", "sessions", "present", "absent"),
            show="headings", height=20
        )
        self.pres_table.heading("mac", text="mac")
        self.pres_table.heading("first", text="first_seen")
        self.pres_table.heading("last", text="last_seen ")
        self.pres_table.heading("sessions", text="sessions")
        self.pres_table.heading("present", text="total_present_time")
        self.pres_table.heading("absent", text="total_absent_time")

        self.pres_table.column("mac", anchor="w", width=220, stretch=False)
        self.pres_table.column("first", anchor="w", width=260, stretch=True)
        self.pres_table.column("last", anchor="w", width=260, stretch=True)
        self.pres_table.column("sessions", anchor="center", width=90, stretch=False)
        self.pres_table.column("present", anchor="center", width=160, stretch=False)
        self.pres_table.column("absent", anchor="center", width=160, stretch=False)

        yscroll = ttk.Scrollbar(box, orient="vertical", command=self.pres_table.yview)
        self.pres_table.configure(yscrollcommand=yscroll.set)

        self.pres_table.pack(side="left", fill="both", expand=True)
        yscroll.pack(side="right", fill="y")

    def _refresh_tab_presence(self, macs):
        self.pres_table.delete(*self.pres_table.get_children())
        dfp = self._compute_presence_summary_filtered(macs)
        for _, r in dfp.iterrows():
            first_disp = r["first_seen"].isoformat() if pd.notna(r["first_seen"]) else ""
            last_disp  = r["last_seen"].isoformat()  if pd.notna(r["last_seen"]) else ""
            self.pres_table.insert(
                "", "end",
                values=(
                    r["mac"],
                    first_disp,
                    last_disp,
                    int(r["number_of_sessions"]),
                    self._fmt_tdelta_hms(r["total_present_time"]),
                    self._fmt_tdelta_hms(r["total_absent_time"]),
                )
            )

    # ---------------------------------------------------------
    # Tab 3: Session Inspector
    # ---------------------------------------------------------
    def _build_tab_session_inspector(self, parent):
        top = ttk.Frame(parent)
        top.pack(fill="x", padx=10, pady=6)

        ttk.Label(top, text="Beacon MAC:").pack(side="left")
        self.mac_var = tk.StringVar()

        self.mac_combo = ttk.Combobox(top, values=[], textvariable=self.mac_var, state="readonly", width=30)
        self.mac_combo.pack(side="left", padx=6)
        self.mac_combo.bind("<<ComboboxSelected>>", self.on_mac_selected)

        self.meta_label = ttk.Label(top, text="", font=("Courier", 10))
        self.meta_label.pack(side="left", padx=18)

        MID= ttk.LabelFrame(parent, text="Sessions")
        MID.pack(fill="both", expand=True, padx=10, pady=6)

        self.session_table = ttk.Treeview(MID,columns=("sid", "start", "end", "duration", "next_gap", "start_utc", "end_utc"),
            show="headings",
            height=9
        )
        col_defs = {
            "sid": ("sid", 140),
            "start": ("start ", 300),
            "end": ("end ", 300),
            "duration": ("duration", 120),
            "next_gap": ("next_gap", 120),
            "start_utc": ("start_utc", 0),
            "end_utc": ("end_utc", 0),
        }
        for c, (label, w) in col_defs.items():
            self.session_table.heading(c, text=label)
            self.session_table.column(c, anchor="center", width=w, stretch=(w > 0))
        self.session_table.column("start_utc", width=0, stretch=False)
        self.session_table.column("end_utc", width=0, stretch=False)

        yscroll = ttk.Scrollbar(MID, orient="vertical", command=self.session_table.yview)
        self.session_table.configure(yscrollcommand=yscroll.set)
        self.session_table.pack(side="left", fill="both", expand=True)
        yscroll.pack(side="right", fill="y")

        self.session_table.bind("<<TreeviewSelect>>", self.on_session_selected)

        bottom = ttk.LabelFrame(parent, text="Session (timestamp → events)")
        bottom.pack(fill="both", expand=True, padx=10, pady=6)

        self.evidence = tk.Text(bottom, wrap="word")
        self.evidence.pack(fill="both", expand=True)

    @staticmethod
    def _fmt_tdelta(td) -> str:
        if td is None or pd.isna(td):
            return ""
        try:
            total_seconds = int(td.total_seconds())
        except Exception:
            return ""
        sign = "-" if total_seconds < 0 else ""
        total_seconds = abs(total_seconds)
        h = total_seconds // 3600
        m = (total_seconds % 3600) // 60
        s = total_seconds % 60
        return f"{sign}{h}:{m:02d}:{s:02d}"

    def _refresh_tab_inspector(self, macs):
        self.mac_combo.configure(values=macs)
        cur = (self.mac_var.get() or "").upper().strip()
        if cur not in macs:
            self.mac_var.set(macs[0] if macs else "")
            cur = self.mac_var.get()

        if cur:
            self.on_mac_selected()
        else:
            self._current_mac = None
            self.meta_label.config(text="")
            self.session_table.delete(*self.session_table.get_children())
            self.evidence.delete("1.0", "end")

    def on_mac_selected(self, event=None):
        mac = (self.mac_var.get() or "").upper().strip()
        self._current_mac = mac

        row = self.df_tuples[self.df_tuples["mac"] == mac]
        if not row.empty:
            r = row.iloc[0]
            self.meta_label.config(text=f"UUID={r[self.uuid_col]} | Major={r['major']} | Minor={r['minor']}")
        else:
            self.meta_label.config(text="")

        self.session_table.delete(*self.session_table.get_children())

        dfm = self.df_sessions[self.df_sessions["mac"] == mac].copy()
        if dfm.empty:
            self.evidence.delete("1.0", "end")
            return

        dfm = dfm.sort_values("session_start").reset_index(drop=True)
        dfm["duration"] = dfm["session_end"] - dfm["session_start"]
        dfm["next_gap"] = dfm["session_start"].shift(-1) - dfm["session_end"]

        for _, r in dfm.iterrows():
            start_utc = r["session_start"]
            end_utc = r["session_end"]
            self.session_table.insert("", "end", values=(
                int(r["session_id"]) if pd.notna(r["session_id"]) else "",
                start_utc.isoformat(),
                end_utc.isoformat(),
                self._fmt_tdelta(r["duration"]),
                self._fmt_tdelta(r["next_gap"]),
                start_utc.isoformat(),
                end_utc.isoformat(),
            ))

        self.evidence.delete("1.0", "end")

    def on_session_selected(self, event=None):
        sel = self.session_table.selection()
        if not sel:
            return

        values = self.session_table.item(sel[0])["values"]
        if len(values) < 7:
            return

        start_utc = pd.to_datetime(values[5], utc=True, errors="coerce")
        end_utc = pd.to_datetime(values[6], utc=True, errors="coerce")
        if pd.isna(start_utc) or pd.isna(end_utc):
            return

        mac = self._current_mac
        self.evidence.delete("1.0", "end")

        mask = (
            self.df_raw["ts_utc"].between(start_utc, end_utc, inclusive="both") &
            self.df_raw["mac_list"].apply(lambda xs: mac in (xs or []))
        )

        for _, r in self.df_raw.loc[mask].sort_values("ts_utc").iterrows():
            ev = self.extract_events_for_mac(r["revelations_clean"], mac)
            if ev:
                self.evidence.insert("end", f"- {r['ts_utc'].isoformat()}\n{ev}\n\n")

    # ---------------------------------------------------------
    # Tab 4: Track Devices
    # ---------------------------------------------------------
    def _build_tab_track_devices(self, parent):
        top = ttk.Frame(parent)
        top.pack(fill="x", padx=10, pady=6)

        ttk.Label(top, text="Beacon MAC:").pack(side="left")
        self.track_mac_var = tk.StringVar()

        self.track_mac_combo = ttk.Combobox(
            top, values=self._list_macs, textvariable=self.track_mac_var, state="readonly", width=30
        )
        self.track_mac_combo.pack(side="left", padx=6)
        self.track_mac_combo.bind("<<ComboboxSelected>>", self.on_track_mac_selected)

        self.track_meta_label = ttk.Label(top, text="", font=("Courier", 10))
        self.track_meta_label.pack(side="left", padx=18)

        paned = ttk.Panedwindow(parent, orient="horizontal")
        paned.pack(fill="both", expand=True, padx=10, pady=6)

        left = ttk.Frame(paned)
        right = ttk.Frame(paned)
        paned.add(left, weight=4)
        paned.add(right, weight=2)

        left_box = ttk.LabelFrame(left, text="Timestamp → RSSI (dBm) → Proximity (NEAR/FAR/MID)")
        left_box.pack(fill="both", expand=True)

        self.track_table = ttk.Treeview(
            left_box, columns=("ts", "rssi", "prox"),
            show="headings", height=18
        )
        self.track_table.heading("ts", text="timestamp ")
        self.track_table.heading("rssi", text="rssi (dBm)")
        self.track_table.heading("prox", text="proximity class")

        self.track_table.column("ts", anchor="w", width=520, stretch=True)
        self.track_table.column("rssi", anchor="center", width=90, stretch=False)
        self.track_table.column("prox", anchor="center", width=160, stretch=False)

        yscroll = ttk.Scrollbar(left_box, orient="vertical", command=self.track_table.yview)
        self.track_table.configure(yscrollcommand=yscroll.set)

        self.track_table.pack(side="left", fill="both", expand=True)
        yscroll.pack(side="right", fill="y")

        right_box = ttk.LabelFrame(right, text="Decision Summary")
        right_box.pack(fill="both", expand=True)

        self.track_summary_text = tk.Text(right_box, wrap="word")
        self.track_summary_text.pack(fill="both", expand=True)
        self.track_summary_text.insert("1.0", "Select a MAC to compute decisions.\n")
        self.track_summary_text.configure(state="disabled")

    def on_track_mac_selected(self, event=None):
        mac = self.track_mac_var.get().upper().strip()

        row = self.df_tuples[self.df_tuples["mac"] == mac]
        if not row.empty:
            r = row.iloc[0]
            self.track_meta_label.config(text=f"UUID={r[self.uuid_col]} | Major={r['major']} | Minor={r['minor']}")
        else:
            self.track_meta_label.config(text="")

        self.track_table.delete(*self.track_table.get_children())
        self.track_summary_text.configure(state="normal")
        self.track_summary_text.delete("1.0", "end")
        self.track_summary_text.configure(state="disabled")

        mask = self.df_raw["mac_list"].apply(lambda xs: mac in (xs or []))
        rows = self.df_raw.loc[mask].sort_values("ts_utc")

        samples = []
        rssi_only = []
        prox_counts = {"NEAR": 0, "FAR": 0, "MID": 0}

        for _, rr in rows.iterrows():
            ev = self.extract_events_for_mac(rr["revelations_clean"], mac)
            text_to_parse = ev if ev else rr["revelations_clean"]

            vals = self._parse_rssi_from_text(text_to_parse)
            for v in vals:
                prox = self._proximity_label(v)
                if prox is None:
                    continue
                samples.append((rr["ts_utc"], v, prox))
                rssi_only.append(v)
                prox_counts[prox] += 1

        for ts, v, prox in samples:
            self.track_table.insert("", "end", values=(ts.isoformat(), int(v), prox))
        total_prox = sum(prox_counts.values())
        prox_major = max(prox_counts.items(), key=lambda kv: kv[1])[0] if total_prox else "MID"

        motion_label, std, median_step, n = self._detect_moving_or_static_mad(rssi_only)

        lines = []
        lines.append(f"MAC: {mac}")
        lines.append("")
        lines.append("1) Proximity Status (NEAR / FAR / MID)")
        lines.append("   Thresholds (RSSI in dBm):")
        lines.append(f"     NEAR      if rssi >= {NEAR_RSSI_THRESHOLD} dBm")
        lines.append(f"     FAR       if rssi <= {FAR_RSSI_THRESHOLD} dBm")
        lines.append(f"     MID       if {FAR_RSSI_THRESHOLD} < rssi < {NEAR_RSSI_THRESHOLD} dBm")
        lines.append("")
        lines.append(f"   Total RSSI samples: {total_prox}")
        lines.append("   Counts:")
        lines.append(f"     NEAR     : {prox_counts['NEAR']}")
        lines.append(f"     MID      : {prox_counts['MID']}")
        lines.append(f"     FAR      : {prox_counts['FAR']}")
        lines.append("")
        lines.append("2) Behavior Decision (MOVING / STATIC) [MAD-based]")
        lines.append("   Metrics:")
        lines.append("     std = 1.4826 * MAD (dB)")
        lines.append("     MAD = median(|RSSI - median(RSSI)|)")
        lines.append("     step = median(|diff(RSSI)|) (dB)")
        lines.append("")
        lines.append("   Thresholds:")
        lines.append(f"     STATIC if std <= {STATIC_STD_MAX}")
        lines.append(f"     MOVING if std >= {MOVING_STD_MIN}")
        lines.append(f"     MID-zone: MOVING if step >= {MOVING_STEP_MIN} else STATIC")
        lines.append("")

        if std is None or median_step is None:
            lines.append(f"   Decision: {motion_label} | samples={n} (not enough data for stable stats)")
        else:
            lines.append(f"   std        : {std:.2f} dB")
            lines.append(f"   step       : {median_step:.2f} dB")
            lines.append(f"   Decision   : {motion_label}")

        lines.append("")
        lines.append("Combined:")
        lines.append(f"  Proximity majority = {prox_major}")
        lines.append(f"  Movement behavior  = {motion_label}")

        self.track_summary_text.configure(state="normal")
        self.track_summary_text.insert("1.0", "\n".join(lines))
        self.track_summary_text.configure(state="disabled")

    # ---------------------------------------------------------
    # Tab 5: Plots
    # ---------------------------------------------------------
    def _build_tab_analytics(self, parent):
        top = ttk.Frame(parent)
        top.pack(fill="x", padx=10, pady=(10, 6))

        ttk.Label(top, text="MAC:").pack(side="left")
        self.analytics_mac_var = tk.StringVar()
        self.analytics_mac_combo = ttk.Combobox(
            top, values=[], textvariable=self.analytics_mac_var, state="readonly", width=30
        )
        self.analytics_mac_combo.pack(side="left", padx=6)
        self.analytics_mac_combo.bind(
            "<<ComboboxSelected>>",
            lambda e: self._refresh_analytics_for_mac(self.analytics_mac_var.get())
        )

        ttk.Button(
            top, text="Refresh",
            command=lambda: self._refresh_analytics_for_mac(self.analytics_mac_var.get())
        ).pack(side="left", padx=6)

        ttk.Button(
            top, text="Export Plot (PNG/JPG/PDF)",
            command=self._export_current_plot_image
        ).pack(side="left", padx=6)

        ttk.Button(
            top, text="Export Plot Data (CSV/XLSX)",
            command=self._export_current_plot_data
        ).pack(side="left", padx=6)

        self.analytics_info = ttk.Label(top, text="")
        self.analytics_info.pack(side="left", padx=12)

        body = ttk.Panedwindow(parent, orient="horizontal")
        body.pack(fill="both", expand=True, padx=10, pady=(0, 10))

        left = ttk.Frame(body)
        right = ttk.Frame(body)
        body.add(left, weight=2)
        body.add(right, weight=5)

        stats_box = ttk.LabelFrame(left, text="Per-device stats")
        stats_box.pack(fill="both", expand=True)

        self.analytics_stats = tk.Text(stats_box, wrap="word")
        self.analytics_stats.pack(fill="both", expand=True)
        self.analytics_stats.insert("1.0", "Select a MAC to view analytics.\n")
        self.analytics_stats.configure(state="disabled")

        self.plot_nb = ttk.Notebook(right)
        self.plot_nb.pack(fill="both", expand=True)

        self._plot_tabs = {}
        for name in [
            "RSSI vs Time",
            "Proximity vs Time",
            "Movement Timeline",
            "RSSI PDF",
            "RSSI CDF",
            "Event Count",
        ]:
            frame = ttk.Frame(self.plot_nb)
            self.plot_nb.add(frame, text=name)
            self._plot_tabs[name] = self._make_plot_canvas(frame)

        # -------------------- PATCH ADDITION (export tab) --------------------
        self.plot_nb.bind("<<NotebookTabChanged>>", self._on_plot_tab_changed)
        self._on_plot_tab_changed()
        # ---------------------------------------------------------------------------

    def _make_plot_canvas(self, parent):
        # Bigger size for Tkinter display
        fig = Figure(figsize=(IEEE_WIDTH, IEEE_HEIGHT), dpi=300)
        ax = fig.add_subplot(111)
        canvas = FigureCanvasTkAgg(fig, master=parent)
        widget = canvas.get_tk_widget()
        widget.pack(fill="both", expand=True)
        return {"fig": fig, "ax": ax, "canvas": canvas}


    def _clear_ax(self, name: str):
        p = self._plot_tabs[name]
        p["fig"].clear()
        ax = p["fig"].add_subplot(111)
        p["ax"] = ax
        return ax, p["canvas"], p["fig"]

    def _refresh_analytics_for_mac(self, mac: str):
        mac = (mac or "").upper().strip()
        if not mac:
            return

        df_s, df_bins = self._build_analytics_dataset(mac)
        self._compute_common_time_range(df_s, df_bins)

        n_rssi = int(df_s.shape[0]) if not df_s.empty else 0
        obs = ""
        if not df_s.empty:
            t0 = df_s["ts_utc"].min()
            t1 = df_s["ts_utc"].max()
            obs = f" | span={t1 - t0}"
        self.analytics_info.config(text=f"RSSI samples={n_rssi}{obs}")

        self._render_analytics_stats(mac, df_s, df_bins)
        self._plot_rssi_time(mac, df_s)
        self._plot_proximity_time(mac, df_s)
        self._plot_movement_timeline(mac, df_s)
        self._plot_pdf(mac, df_s)
        self._plot_cdf(mac, df_s)
        self._plot_event_count(mac, df_bins)
        self._on_plot_tab_changed()

    def _render_analytics_stats(self, mac: str, df_s: pd.DataFrame, df_bins: pd.DataFrame):
        lines = []
        lines.append(f"MAC: {mac}")
        lines.append(f"Event binning: {self.EVENT_BIN_LABEL} | Rolling window: {ROLLING_WINDOW} samples")
        lines.append("")

        if df_s.empty:
            lines.append("No RSSI samples found for this MAC.")
        else:
            r = df_s["rssi"].astype(float).values
            mean = float(np.mean(r))
            med = float(np.median(r))
            std = float(np.std(r, ddof=1)) if r.size >= 2 else 0.0
            mad_std = float(self._mad_std(r))
            p10, p50, p90 = np.percentile(r, [10, 50, 90])

            prox_counts = df_s["prox"].value_counts().to_dict()
            near = int(prox_counts.get("NEAR", 0))
            MID  = int(prox_counts.get("MID", 0))
            far  = int(prox_counts.get("FAR", 0))
            total = int(r.size)

            lines.append("RSSI Statistics (dBm):")
            lines.append(f"  N samples     : {total}")
            lines.append(f"  mean          : {mean:.2f} dBm")
            lines.append(f"  median        : {med:.2f} dBm")
            lines.append(f"  std (classic) : {std:.2f} dB")
            lines.append(f"  std (MAD)     : {mad_std:.2f} dB")
            lines.append(f"  p10/p50/p90   : {p10:.1f} / {p50:.1f} / {p90:.1f} dBm")
            lines.append("")
            lines.append("Proximity breakdown:")
            lines.append(f"  NEAR      : {near}  ({(near/total*100):.1f}%)")
            lines.append(f"  MID : {MID}   ({(MID/total*100):.1f}%)")
            lines.append(f"  FAR       : {far}   ({(far/total*100):.1f}%)")

        if not df_bins.empty and {"NEW", "CHG", "DEL"}.issubset(df_bins.columns):
            lines.append("")
            lines.append("Event Totals (MAC-scoped):")
            lines.append(f"  NEW={int(df_bins['NEW'].sum())}, CHG={int(df_bins['CHG'].sum())}, DEL={int(df_bins['DEL'].sum())}")

        self.analytics_stats.configure(state="normal")
        self.analytics_stats.delete("1.0", "end")
        self.analytics_stats.insert("1.0", "\n".join(lines))
        self.analytics_stats.configure(state="disabled")

    # ---------------------------------------------------------
    # Plots consistent legend style
    # ---------------------------------------------------------
    def _apply_time_axis_style(self, ax):
        ax.set_xlabel(self.TIME_AXIS_LABEL)
        ax.xaxis.set_major_formatter(mdates.DateFormatter(self.TIME_TICK_FMT, tz=UTC_TZ))
        for lbl in ax.get_xticklabels():
            lbl.set_rotation(30)
            lbl.set_ha("right")
        ax.tick_params(axis="x", pad=8)

    def _apply_legend_style(self, ax):
        leg = ax.legend(**self.LEGEND_KW)
        if leg is not None:
            leg.set_title("")  # keep consistent (no legend titles)
        return leg
        

    def _plot_rssi_time(self, mac: str, df_s: pd.DataFrame):
        ax, canvas, _ = self._clear_ax("RSSI vs Time")
        ax.set_title(f"RSSI over time — {mac}")
    
        # Empty export schema (prevents reusing previous plot df)
        if df_s is None or df_s.empty or "ts_utc" not in df_s.columns or "rssi" not in df_s.columns:
            empty = pd.DataFrame(columns=["ts_utc", "rssi"])
            self._set_current_plot_context("RSSI vs Time", mac, empty)
    
            ax.text(0.5, 0.5, "No RSSI samples.", ha="center", va="center")
            ax.set_xlabel(self.TIME_AXIS_LABEL)
            ax.set_ylabel("RSSI (dBm)")
            self._apply_universal_axis_style(ax)
            ax.figure.tight_layout()
            canvas.draw()
            return
    
        d = df_s.sort_values("ts_utc").copy()
        ts_utc = pd.to_datetime(d["ts_utc"], utc=True, errors="coerce")
        d = d.loc[ts_utc.notna()].copy()
        ts_utc = pd.to_datetime(d["ts_utc"], utc=True, errors="coerce")


        # Plot (UTC tz-aware)
        x_plot = ts_utc
        y_plot = d["rssi"].astype(float)
        
        ax.plot(x_plot, y_plot, marker="o", linestyle="none", markersize=3)
        self._apply_time_axis_style(ax)
        self._apply_common_time_xlim(ax)
        ax.set_ylabel("RSSI (dBm)")
        self._apply_universal_axis_style(ax)
        
        # Export ONLY UTC
        df_export = pd.DataFrame({
            "ts_utc": ts_utc,
            "rssi": y_plot.values,
        })
        self._set_current_plot_context("RSSI vs Time", mac, df_export)
        ax.figure.tight_layout()
        canvas.draw()

    
    def _plot_proximity_time(self, mac: str, df_s: pd.DataFrame):
        ax, canvas, _ = self._clear_ax("Proximity vs Time")
        ax.set_title(f"Proximity over time — {mac}")
    
        export_cols = ["ts_utc", "rssi", "prox", "prox_y"]
    
        # ---- guard / empty ----
        if (
            df_s is None or df_s.empty
            or "ts_utc" not in df_s.columns
            or "prox" not in df_s.columns
            or "rssi" not in df_s.columns
        ):
            empty = pd.DataFrame(columns=export_cols)
            self._set_current_plot_context("Proximity vs Time", mac, empty)
    
            ax.text(0.5, 0.5, "No RSSI samples.", ha="center", va="center")
            ax.set_xlabel(self.TIME_AXIS_LABEL)
            ax.set_ylabel("Proximity class")
            self._apply_universal_axis_style(ax)
            ax.figure.tight_layout()
            canvas.draw()
            return
    
        # ---- clean ----
        d = df_s.sort_values("ts_utc").copy()
        d["ts_utc"] = pd.to_datetime(d["ts_utc"], utc=True, errors="coerce")
        d["rssi"] = pd.to_numeric(d["rssi"], errors="coerce")
        d["prox"] = d["prox"].astype(str).str.strip().str.upper()
    
        # normalize possible variants
        d["prox"] = d["prox"].replace({"MID RANGE": "MID", "MID-RANGE": "MID"})
    
        d = d.dropna(subset=["ts_utc", "rssi", "prox"]).copy()
        if d.empty:
            empty = pd.DataFrame(columns=export_cols)
            self._set_current_plot_context("Proximity vs Time", mac, empty)
    
            ax.text(0.5, 0.5, "No valid RSSI/proximity data.", ha="center", va="center")
            ax.set_xlabel(self.TIME_AXIS_LABEL)
            ax.set_ylabel("Proximity class")
            self._apply_universal_axis_style(ax)
            ax.figure.tight_layout()
            canvas.draw()
            return
    
        # ---- map proximity to numeric y (categorical) ----
        prox_map = {"FAR": 0, "MID": 1, "NEAR": 2}
        d["prox_y"] = d["prox"].map(prox_map)
        d = d.dropna(subset=["prox_y"]).copy()
        if d.empty:
            empty = pd.DataFrame(columns=export_cols)
            self._set_current_plot_context("Proximity vs Time", mac, empty)
    
            ax.text(0.5, 0.5, "No proximity labels available.", ha="center", va="center")
            ax.set_xlabel(self.TIME_AXIS_LABEL)
            # ax.set_ylabel("Proximity class")
            self._apply_universal_axis_style(ax)
            ax.figure.tight_layout()
            canvas.draw()
            return
    
        d["prox_y"] = d["prox_y"].astype(int)
    
        ts_utc = d["ts_utc"]
        x_plot = ts_utc    
        # ---- plot proximity points (left axis) ----
        ax.plot(x_plot, d["prox_y"], marker="o", linestyle="none", markersize=3)
    
        self._apply_time_axis_style(ax)
        self._apply_common_time_xlim(ax)
        # ax.set_ylabel("Proximity class")
        ax.set_yticks([0, 1, 2])
        ax.set_yticklabels(["FAR", "MID", "NEAR"])
        ax.set_ylim(-0.25, 2.25) 
        self._apply_universal_axis_style(ax)
    
        # =========================================================
        # Right-side axis: RSSI (dBm)
        # =========================================================
        ax2 = ax.twinx()
    
        # Plot RSSI points on the right axis 
        ax2.scatter(x_plot, d["rssi"].astype(float), s=12)
    
        # Use a stable RSSI range; fall back to data-driven if needed
        stable_low, stable_high = -100.0, -40.0
        rmin = float(np.nanmin(d["rssi"].values))
        rmax = float(np.nanmax(d["rssi"].values))
        if np.isfinite(rmin) and np.isfinite(rmax):
            # If your data is wildly outside BLE RSSI, adapt; otherwise stable view
            if rmin < stable_low or rmax > stable_high:
                pad = 2.0
                ax2.set_ylim(rmin - pad, rmax + pad)
            else:
                ax2.set_ylim(stable_low, stable_high)
        else:
            ax2.set_ylim(stable_low, stable_high)
    
        ax2.set_ylabel("RSSI (dBm)")
        ax2.tick_params(axis="y", labelsize=self.PLOT_STYLE["tick_size"])
        ax2.yaxis.label.set_fontsize(self.PLOT_STYLE["label_size"])
        ax2.tick_params(axis="both", which="major", labelsize=self.PLOT_STYLE["tick_size"])
        ax2.tick_params(axis="both", which="minor", labelsize=self.PLOT_STYLE["tick_size"])
        
    
        # show threshold lines on RSSI axis for visual validation
        # (NEAR >= -50, FAR <= -75)
        try:
            ax2.axhline(NEAR_RSSI_THRESHOLD, linewidth=1.4, alpha=0.35)
            ax2.axhline(FAR_RSSI_THRESHOLD, linewidth=1.4, alpha=0.35)
        except Exception:
            pass
    
        # ---- export ----
        df_export = pd.DataFrame({
            "ts_utc": ts_utc,
            "rssi": d["rssi"].astype(float).values,
            "prox": d["prox"].astype(str).values,
            "prox_y": d["prox_y"].astype(int).values,
        })
        self._set_current_plot_context("Proximity vs Time", mac, df_export)
        ax.figure.tight_layout()    
        canvas.draw()

    def _plot_movement_timeline(self, mac: str, df_s: pd.DataFrame):
        ax, canvas, fig = self._clear_ax("Movement Timeline")
        ax.set_title(f"Movement indicator over time — {mac}")
    
        export_cols = ["ts_utc", "roll_mad_std", "roll_step", "label", "is_moving"]
    
        # -------------------- guards --------------------
        if df_s is None or df_s.empty or "ts_utc" not in df_s.columns or "rssi" not in df_s.columns:
            empty = pd.DataFrame(columns=export_cols)
            self._set_current_plot_context("Movement Timeline", mac, empty)
    
            ax.text(0.5, 0.5, "No RSSI samples.", ha="center", va="center")
            ax.set_ylabel("MAD-std (dB)")
    
            # align time axis style with other plots
            self._apply_time_axis_style(ax)
            self._apply_common_time_xlim(ax)
    
            self._apply_universal_axis_style(ax)
            ax.figure.tight_layout()
            canvas.draw()
            return
    
        d = df_s.sort_values("ts_utc").copy()
        ts_utc = pd.to_datetime(d["ts_utc"], utc=True, errors="coerce")
        d = d.loc[ts_utc.notna()].copy()
        ts_utc = pd.to_datetime(d["ts_utc"], utc=True, errors="coerce")

        if d["rssi"].dropna().empty:
            empty = pd.DataFrame(columns=export_cols)
            self._set_current_plot_context("Movement Timeline", mac, empty)
        
            ax.text(0.5, 0.5, "No RSSI samples.", ha="center", va="center")
            ax.set_ylabel("MAD-std (dB)")
        
            # align time axis style with other plots
            self._apply_time_axis_style(ax)
            self._apply_common_time_xlim(ax)
        
            self._apply_universal_axis_style(ax)
            ax.figure.tight_layout()
            canvas.draw()
            return
    
        # -------------------- rolling stats --------------------
        s = pd.Series(d["rssi"].astype(float).values, index=ts_utc)
        mov = self._rolling_movement(s, window=ROLLING_WINDOW)
    
        if mov.empty or mov["label"].dropna().empty:
            empty = pd.DataFrame(columns=export_cols)
            self._set_current_plot_context("Movement Timeline", mac, empty)
    
            ax.text(0.5, 0.5, f"Not enough data for rolling window={ROLLING_WINDOW}.",
                    ha="center", va="center")
            ax.set_ylabel("MAD-std (dB)")
    
            # align time axis style with other plots
            self._apply_time_axis_style(ax)
            self._apply_common_time_xlim(ax)
    
            self._apply_universal_axis_style(ax)
            ax.figure.tight_layout()
            canvas.draw()
            return

        def _med_step(x):
            x = np.array(x, dtype=float)
            if x.size < 2:
                return 0.0
            return float(np.median(np.abs(np.diff(x))))
    
        roll_step = s.rolling(ROLLING_WINDOW).apply(_med_step, raw=False).reindex(mov.index)
    
        # -------------------- plotting --------------------
        mov_plot = mov.copy()
        mov_plot = mov_plot[~mov_plot.index.duplicated(keep="last")]
    
        ts_idx_plot = pd.DatetimeIndex(mov_plot.index)
        x_plot = ts_idx_plot
    
        ax.step(
            x_plot,
            mov_plot["roll_mad_std"].astype(float),
            where="post",
            label="RSSI spread (MAD-based)",
            linewidth=1.4,
        )
    
        ax.axhline(STATIC_STD_MAX, linestyle="--", linewidth=1.0, alpha=0.6,
                   label=f"STATIC ≤ {STATIC_STD_MAX:g} dB")
        ax.axhline(MOVING_STD_MIN, linestyle="--", linewidth=1.0, alpha=0.6,
                   label=f"MOVING ≥ {MOVING_STD_MIN:g} dB")
    
        labels_plot = mov_plot["label"].fillna("STATIC").astype(str).values
        moving_mask = np.array([1 if v == "MOVING" else 0 for v in labels_plot], dtype=int)
    
        first_span = True
        in_seg = False
        start = 0
        for i, flag in enumerate(moving_mask):
            if flag == 1 and not in_seg:
                in_seg = True
                start = i
            if in_seg and (flag == 0 or i == len(moving_mask) - 1):
                end = (i - 1) if flag == 0 else i
                if end > start:
                    ax.axvspan(
                        x_plot[start],
                        x_plot[end],
                        alpha=0.20,
                        zorder=0,
                        label="MOVING interval" if first_span else None,
                    )
                    first_span = False
                in_seg = False
    
        y = mov_plot["roll_mad_std"].astype(float).values
        if np.isfinite(y).any():
            ymin = float(np.nanmin(y))
            ymax = float(np.nanmax(y))
            ax.set_ylim(max(0.0, ymin - 0.2), ymax + 0.2)
    
        # -------------------- styling --------------------
        ax.set_ylabel("MAD-std (dB)")
        ax.grid(False)
    
        self._apply_time_axis_style(ax)
        self._apply_common_time_xlim(ax)
        self._apply_legend_style(ax)
    
        self._apply_universal_axis_style(ax)
    
        # -------------------- export --------------------
        ts_idx = pd.DatetimeIndex(mov.index)
        df_export = pd.DataFrame({
            "ts_utc": ts_idx,
            "roll_mad_std": mov["roll_mad_std"].astype(float).values,
            "roll_step": roll_step.astype(float).values,
            "label": mov["label"].fillna("STATIC").astype(str).values,
            "is_moving": (mov["label"].fillna("STATIC").astype(str).values == "MOVING"),
        })
        self._set_current_plot_context("Movement Timeline", mac, df_export)
        ax.figure.tight_layout()
        canvas.draw()


    def _plot_pdf(self, mac: str, df_s: pd.DataFrame):
        ax, canvas, _ = self._clear_ax("RSSI PDF")
        ax.set_title(f"RSSI distribution (PDF) — {mac}")
    
        export_cols = [
            "hist_bin_left", "hist_bin_right", "hist_density",
            "fit_x", "fit_pdf", "mu_dBm", "sigma_dB"
        ]
    
        if df_s is None or df_s.empty or "rssi" not in df_s.columns:
            empty = pd.DataFrame(columns=export_cols)
            self._set_current_plot_context("RSSI PDF", mac, empty)
    
            ax.text(0.5, 0.5, "No RSSI samples.", ha="center", va="center")
            ax.set_xlabel("RSSI (dBm)")
            ax.set_ylabel("Probability density (1/dBm)")
            self._apply_universal_axis_style(ax)
            ax.figure.tight_layout()
            canvas.draw()
            return
    
        r = df_s["rssi"].astype(float).dropna().values
        if r.size == 0:
            empty = pd.DataFrame(columns=export_cols)
            self._set_current_plot_context("RSSI PDF", mac, empty)
    
            ax.text(0.5, 0.5, "No RSSI samples.", ha="center", va="center")
            ax.set_xlabel("RSSI (dBm)")
            ax.set_ylabel("Probability density (1/dBm)")
            self._apply_universal_axis_style(ax)
            ax.figure.tight_layout()
            canvas.draw()
            return
    
        # -------- Histogram --------
        bins = 25
        hist_density, bin_edges = np.histogram(r, bins=bins, density=True)
        bin_left = bin_edges[:-1]
        bin_right = bin_edges[1:]
    
        ax.hist(
            r,
            bins=bins,
            density=True,
            alpha=0.65,
            edgecolor="black",
            linewidth=2.8
        )
    
        # -------- Normal Fit --------
        mu = float(np.mean(r))
        sigma = float(np.std(r, ddof=1)) if r.size >= 2 else 0.0
    
        fit_x = np.linspace(r.min(), r.max(), 300)
        fit_pdf = np.full_like(fit_x, np.nan, dtype=float)
    
        if sigma > 0:
            fit_pdf = (
                1.0 / (sigma * np.sqrt(2 * np.pi))
            ) * np.exp(-0.5 * ((fit_x - mu) / sigma) ** 2)
    
            ax.plot(
                fit_x,
                fit_pdf,
                linewidth=1.4,
                label=f"Normal fit (μ={mu:.1f} dBm, σ={sigma:.1f} dB)"
            )
    
            self._apply_legend_style(ax)
    
        # -------- Styling --------
        ax.set_xlabel("RSSI (dBm)")
        ax.set_ylabel("Probability density (1/dBm)")
        ax.grid(True, alpha=0.2)
        self._apply_universal_axis_style(ax)
    
        # -------- Export --------
        n_hist = hist_density.size
        n_fit = fit_x.size
        n = max(n_hist, n_fit)
    
        def _pad(a, n):
            out = np.full(n, np.nan, dtype=float)
            out[:len(a)] = a
            return out
    
        df_export = pd.DataFrame({
            "hist_bin_left": _pad(bin_left, n),
            "hist_bin_right": _pad(bin_right, n),
            "hist_density": _pad(hist_density, n),
            "fit_x": _pad(fit_x, n),
            "fit_pdf": _pad(fit_pdf, n),
            "mu_dBm": np.full(n, mu, dtype=float),
            "sigma_dB": np.full(n, sigma, dtype=float),
        })
    
        self._set_current_plot_context("RSSI PDF", mac, df_export)
        ax.figure.tight_layout()
        canvas.draw()


    def _plot_cdf(self, mac: str, df_s: pd.DataFrame):
        ax, canvas, _ = self._clear_ax("RSSI CDF")
        ax.set_title(f"RSSI cumulative distribution (CDF) — {mac}")
    
        export_cols = ["rssi_sorted_dBm", "cdf"]
    
        if df_s is None or df_s.empty or "rssi" not in df_s.columns:
            empty = pd.DataFrame(columns=export_cols)
            self._set_current_plot_context("RSSI CDF", mac, empty)
    
            ax.text(0.5, 0.5, "No RSSI samples.", ha="center", va="center")
            ax.set_xlabel("RSSI (dBm)")
            ax.set_ylabel("CDF")
            self._apply_universal_axis_style(ax)
            ax.figure.tight_layout()
            canvas.draw()
            return
    
        r = df_s["rssi"].astype(float).dropna().values
        if r.size == 0:
            empty = pd.DataFrame(columns=export_cols)
            self._set_current_plot_context("RSSI CDF", mac, empty)
    
            ax.text(0.5, 0.5, "No RSSI samples.", ha="center", va="center")
            ax.set_xlabel("RSSI (dBm)")
            ax.set_ylabel("CDF")
            self._apply_universal_axis_style(ax)
            ax.figure.tight_layout()
            canvas.draw()
            return
    
        # --- CDF computation ---
        r_sorted = np.sort(r)
        y = np.arange(1, r_sorted.size + 1) / float(r_sorted.size)
    
        # --- Use step plot ---
        ax.step(r_sorted, y, where="post", linewidth=1.4)
    
        # --- Clean axis limits ---
        ax.set_ylim(0, 1.02)
        ax.set_xlim(r_sorted.min(), r_sorted.max())
    
        ax.set_xlabel("RSSI (dBm)")
        ax.set_ylabel("CDF")
        ax.grid(False)
    
        self._apply_universal_axis_style(ax)
    
        df_export = pd.DataFrame({
            "rssi_sorted_dBm": r_sorted,
            "cdf": y,
        })
        self._set_current_plot_context("RSSI CDF", mac, df_export)
        ax.figure.tight_layout()
        canvas.draw()


    def _plot_event_count(self, mac: str, df_bins: pd.DataFrame):
        ax, canvas, _ = self._clear_ax("Event Count")
        ax.set_title(f"Advertisement event count (Δt = {self.EVENT_BIN_LABEL}) — {mac}")
    
        export_cols = ["ts_utc", "NEW", "CHG", "DEL"]
    
        if (
            df_bins is None or df_bins.empty
            or "ts_utc" not in df_bins.columns
            or not {"NEW", "CHG", "DEL"}.issubset(df_bins.columns)
        ):
            empty = pd.DataFrame(columns=export_cols)
            self._set_current_plot_context("Event Count", mac, empty)
    
            ax.text(0.5, 0.5, "No MAC-scoped NEW/CHG/DEL events found.", ha="center", va="center")
            ax.set_xlabel(self.TIME_AXIS_LABEL)
            ax.set_ylabel(f"Events / {self.EVENT_BIN_LABEL}")
            self._apply_universal_axis_style(ax)
            ax.figure.tight_layout()
            canvas.draw()
            return
    
        d = df_bins.sort_values("ts_utc").copy()
        ts_utc = pd.to_datetime(d["ts_utc"], utc=True, errors="coerce")
        d = d.loc[ts_utc.notna()].copy()
        ts_utc = pd.to_datetime(d["ts_utc"], utc=True, errors="coerce")
    
        x_plot = ts_utc     
        ax.plot(x_plot, d["NEW"].astype(float), label="NEW", linewidth=1.4)
        ax.plot(x_plot, d["CHG"].astype(float), label="CHG", linewidth=1.4)
        ax.plot(x_plot, d["DEL"].astype(float), label="DEL", linewidth=1.4)
    
        self._apply_time_axis_style(ax)
        self._apply_common_time_xlim(ax)
        ax.set_ylabel(f"Events / {self.EVENT_BIN_LABEL}")
        self._apply_legend_style(ax)
        self._apply_universal_axis_style(ax)
    
        df_export = pd.DataFrame({
            "ts_utc": ts_utc,
            "NEW": d["NEW"].astype(int).values,
            "CHG": d["CHG"].astype(int).values,
            "DEL": d["DEL"].astype(int).values,
        })
        self._set_current_plot_context("Event Count", mac, df_export)
        ax.figure.tight_layout()
        canvas.draw()


   


    # ---------------------------------------------------------
    # Refresh all tabs when filter changes
    # ---------------------------------------------------------
    def _refresh_all_tabs(self):
        macs = self._filtered_macs()
    
        self.filter_status.config(
            text=f"Active filter: {self._filter_text!r}  | matches: {len(macs)}"
            if self._filter_text else f"(no filter) | MACs: {len(macs)}"
        )
    
        # ---------------------------------------------------------
        # IMPORTANT: If no MACs match filter → reset everything
        # ---------------------------------------------------------
        if not macs:
            # --- Clear iBeacon tab ---
            self.iBeacon_text.configure(state="normal")
            self.iBeacon_text.delete("1.0", "end")
            self.iBeacon_text.insert("1.0", "No MACs match the current filter.\n")
            self.iBeacon_text.configure(state="disabled")
            self.inv_table.delete(*self.inv_table.get_children())
    
            # --- Clear Presence tab ---
            self.pres_table.delete(*self.pres_table.get_children())
    
            # --- Clear Session Inspector ---
            self.mac_var.set("")
            self.session_table.delete(*self.session_table.get_children())
            self.evidence.delete("1.0", "end")
            self.meta_label.config(text="")
    
            # --- Clear Track Devices ---
            self.track_mac_var.set("")
            self.track_table.delete(*self.track_table.get_children())
            self.track_summary_text.configure(state="normal")
            self.track_summary_text.delete("1.0", "end")
            self.track_summary_text.insert("1.0", "No MACs match the current filter.\n")
            self.track_summary_text.configure(state="disabled")
    
            # --- Clear Analytics ---
            self.analytics_mac_var.set("")
            self.analytics_info.config(text="")
            self.analytics_stats.configure(state="normal")
            self.analytics_stats.delete("1.0", "end")
            self.analytics_stats.insert("1.0", "No MAC selected.\n")
            self.analytics_stats.configure(state="disabled")
    
            # --- VERY IMPORTANT: Clear export context ---
            self._current_plot_name = None
            self._current_plot_mac = None
            self._current_plot_df = None
            self._plot_data.clear()
    
            return  # stop here
    
        # ---------------------------------------------------------
        # Normal refresh (when MACs exist)
        # ---------------------------------------------------------
        self._refresh_tab_ibeacon(macs)
        self._refresh_tab_presence(macs)
        self._refresh_tab_inspector(macs)
    
        # --- Track Devices ---
        self.track_mac_combo.configure(values=macs)
        cur4 = (self.track_mac_var.get() or "").upper().strip()
        if cur4 not in macs:
            self.track_mac_var.set(macs[0])
            cur4 = macs[0]
        if cur4:
            self.on_track_mac_selected()
    
        # --- Analytics ---
        self.analytics_mac_combo.configure(values=macs)
        cur5 = (self.analytics_mac_var.get() or "").upper().strip()
        if cur5 not in macs:
            self.analytics_mac_var.set(macs[0])
            cur5 = macs[0]
    
        if cur5:
            self._refresh_analytics_for_mac(cur5)
        else:
            self.analytics_info.config(text="")
            self.analytics_stats.configure(state="normal")
            self.analytics_stats.delete("1.0", "end")
            self.analytics_stats.insert("1.0", "No MAC selected.\n")
            self.analytics_stats.configure(state="disabled")

app = BeaconDeepViewer(
    mac_beacon_df,
    df_beacon_session_detail,
    df
)
app.mainloop()